## In this notebook, we assume users are using GPU-enabled environment <br>(T4 on Google Colab is enough)
- consider run this file on Google Colab
- requirements.txt was prepared for environment on Google Colab

In [ ]:
import os
os.getcwd()

'/content'

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
!git clone https://github.com/slab-it/FRATTVAE.git

fatal: destination path 'FRATTVAE' already exists and is not an empty directory.


## installing libraries with pip

Be careful. Here, downgrading scipy causes restarting session on Google Colab due to Numpy's dependancy(Numpy 2.0 ->Numpy.1.11.3)

In [ ]:
!pip install scipy==1.11.3

In [ ]:

!pip install torch==2.2.0 torchvision==0.17.0 torchaudio==2.2.0 --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118


In [ ]:

!pip install  dgl -f https://data.dgl.ai/wheels/torch-2.2/cu118/repo.html

Looking in links: https://data.dgl.ai/wheels/torch-2.2/cu118/repo.html


In [ ]:
# !pip install rdkit==2023.9.1
!pip install rdkit

In [ ]:
!pip install timeout-decorator

In [ ]:
!pip install molvs

In [ ]:
!pip3 install fcd-torch guacamol

In [ ]:
!pip install botorch

In [ ]:
!pip install molsets

  Using cached molsets-0.3.1-py3-none-any.whl.metadata (581 bytes)
  Using cached pomegranate-0.12.0.tar.gz (3.3 MB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
!pip install gpu4pyscf-cuda12x
!pip install pyscf
!pip install geometric


In [ ]:
!pip freeze > requirements.txt

# In this section, codes were cited from https://github.com/slab-it/FRATTVAE

## scripts/utils/apps.py




In [ ]:
import random
import numpy as np
import pandas as pd
import torch

def torch_fix_seed(seed= 42):
    # Python random
    random.seed(seed)
    # Numpy
    np.random.seed(seed)
    # Pytorch
    torch.manual_seed(seed)
    torch.use_deterministic_algorithms = True
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.backends.cudnn.deterministic = True

def second2date(seconds: int) -> str:
    seconds = int(seconds)
    d = seconds // (3600*24)
    h = (seconds - 3600*24*d) // 3600
    m = (seconds - 3600*24*d - 3600*h) // 60
    s = seconds - 3600*24*d - 3600*h - 60*m

    return f'{d:0>2}:{h:0>2}:{m:0>2}:{s:0>2}'

def list2pdData(loss_list: list, metrics: list) -> pd.DataFrame:
    loss_dict = {}
    for key, values in zip(metrics, loss_list):
        loss_dict[key] = values
    return pd.DataFrame(loss_dict)

## scripts/utils/chem_metrics.py

In [ ]:
from typing import List

import random
import numpy as np
from rdkit import Chem
# from moses import metrics
import networkx as nx

from guacamol.distribution_matching_generator import DistributionMatchingGenerator
from guacamol.distribution_learning_benchmark import KLDivBenchmark
from guacamol.frechet_benchmark import FrechetBenchmark


def normalize(props: np.ndarray, pname: str):
    if pname not in NORM_PARAMS.keys():
        return props.tolist()
    else:
        low, high = NORM_PARAMS[pname]
        return (props - low) / (high - low)


def get_all_metrics(mol) -> dict:
    if type(mol) == str:
        mol = Chem.MolFromSmiles(mol)
    if mol is None:
        return [float('nan')] * len(METRICS_DICT)
    return [func(mol) for func in METRICS_DICT.values()]


def get_metrics(mol, key: str) -> dict:
    if type(mol) == str:
        mol = Chem.MolFromSmiles(mol)
    if mol is None:
        return float('nan')
    return METRICS_DICT[key](mol)


class MockGenerator(DistributionMatchingGenerator):
    """
    Mock generator that returns pre-defined molecules,
    possibly split in several calls
    """
    def __init__(self, molecules: List[str]) -> None:
        self.molecules = molecules
        self.cursor = 0

    def generate(self, number_samples: int) -> List[str]:
        end = self.cursor + number_samples

        sampled_molecules = self.molecules[self.cursor:end]
        self.cursor = end
        return sampled_molecules


def physchem_divergence(smiles: list, reference: list, seed: int= 0):
    """
    smiles: list of generated smiles
    reference: list of training/test smiles

    return average physchem kl-divergence
    """
    generator = MockGenerator(smiles)
    if len(reference) < len(smiles):
        random.seed(seed)
        reference += random.choices(reference, k= len(smiles)-len(reference))
    benchmark = KLDivBenchmark(number_samples= len(smiles), training_set= reference)

    return benchmark.assess_model(generator).score


def guacamol_fcd(smiles: list, reference: list, seed: int= 0):
    """
    smiles: list of generated smiles
    reference: list of training/test smiles

    rerurn standardize fcd [exp(-0.2 * fcd)]
    """
    generator = MockGenerator(smiles)
    if len(reference) < len(smiles):
        random.seed(seed)
        reference += random.choices(reference, k= len(smiles)-len(reference))
    benchmark = FrechetBenchmark(sample_size= len(smiles), training_set= reference)
    try:
        score = benchmark.assess_model(generator).score
        return score
    except Exception as e:
        print(f'[WARNING] can\'t calculate fcd score because of {e}...', flush= True)
        return float('nan')


def penalized_logp(mol):
    """
    Reward that consists of log p penalized by SA and # long cycles,
    as described in (Kusner et al. 2017). Scores are normalized based on the
    statistics of 250k_rndm_zinc_drugs_clean.smi dataset
    :param mol: rdkit mol object
    :return: float

    Reference: 'https://github.com/bowenliu16/rl_graph_generation/blob/master/gym-molecule/gym_molecule/envs/molecule.py'
    """
    # normalization constants, statistics from 250k_rndm_zinc_drugs_clean.smi
    logP_mean = 2.4570953396190123
    logP_std = 1.434324401111988
    SA_mean = -3.0525811293166134
    SA_std = 0.8335207024513095
    cycle_mean = -0.0485696876403053
    cycle_std = 0.2860212110245455

    log_p = metrics.logP(mol)
    SA = -metrics.SA(mol)

    # cycle score
    cycle_list = nx.cycle_basis(nx.Graph(
        Chem.rdmolops.GetAdjacencyMatrix(mol)))
    if len(cycle_list) == 0:
        cycle_length = 0
    else:
        cycle_length = max([len(j) for j in cycle_list])
    if cycle_length <= 6:
        cycle_length = 0
    else:
        cycle_length = cycle_length - 6
    cycle_score = -cycle_length

    normalized_log_p = (log_p - logP_mean) / logP_std
    normalized_SA = (SA - SA_mean) / SA_std
    normalized_cycle = (cycle_score - cycle_mean) / cycle_std

    return normalized_log_p + normalized_SA + normalized_cycle


NORM_PARAMS = {'MW': [0, 1000],
               'logP': [-10, 10],
               'QED': [0, 1],
               'SA': [10, 0],
               'NP': [-5, 5],
               'TPSA': [0, 300],
            #    'BertzCT': [0, 2000],
            #    'PlogP': [-20, 0]
               }

# METRICS_DICT = {'MW': metrics.weight,
#                 'logP': metrics.logP,
#                 'QED': metrics.QED,
#                 'SA': metrics.SA,
#                 'NP': metrics.NP,
#                 'TPSA': Chem.Descriptors.TPSA,
#                 # 'BertzCT': Chem.Descriptors.BertzCT,
#                 # 'PlogP': penalized_logp
#                 }

## scripts/utils/construct.py

In [ ]:
import numpy as np
from copy import deepcopy
from itertools import chain

import timeout_decorator

from rdkit import Chem, rdBase, RDLogger
from rdkit.Chem import RWMol, AllChem, DataStructs
from rdkit.Chem.EnumerateStereoisomers import EnumerateStereoisomers, StereoEnumerationOptions
# from molvs import Standardizer

BONDTYPES = {1: Chem.rdchem.BondType.SINGLE,
             2: Chem.rdchem.BondType.DOUBLE,
             3: Chem.rdchem.BondType.TRIPLE}


def isomer_search(smiles, ecfp: np.ndarray, radius: int= 2):
    if type(ecfp) == list:
        ecfp = np.array(ecfp)
    if smiles is None:
        return None
    mol = Chem.MolFromSmiles(smiles)
    n_bits = len(ecfp)
    opts = StereoEnumerationOptions(unique= True, onlyUnassigned= True)
    isomers = list(EnumerateStereoisomers(mol, options=opts))
    if len(isomers) == 0:
        return smiles
    else:
        isomers += [mol]
    euclids = []
    for isomer in isomers:
        iso_ecfp = np.array(AllChem.GetMorganFingerprintAsBitVect(isomer, radius, n_bits, useChirality= True))
        euclids.append(np.linalg.norm(ecfp-iso_ecfp))

    return Chem.MolToSmiles(isomers[np.argmin(euclids)])


def calc_tanimoto(smi1, smi2, useChiral: bool= True):
    if (smi1 is None) | (smi2 is None):
        return float('nan')
    ecfp1 = AllChem.GetMorganFingerprintAsBitVect(Chem.MolFromSmiles(smi1), 2, 2048, useChirality= useChiral)
    ecfp2 = AllChem.GetMorganFingerprintAsBitVect(Chem.MolFromSmiles(smi2), 2, 2048, useChirality= useChiral)
    return DataStructs.TanimotoSimilarity(ecfp1, ecfp2)


def reconstructMol(smiles, frags, adj, ecfp: list= None,
                   useChiral: bool= True, verbose: bool= False):
    # hasChiral = bool(Chem.FindMolChiralCenters(Chem.MolFromSmiles(smiles))) if useChiral else False
    hasChiral = bool(len(Chem.FindPotentialStereo(Chem.MolFromSmiles(smiles), flagPossible= False))) if useChiral else False
    try:
        # rev_smiles = MolFromFragments(frags, adj, asMol= False, useChiral= hasChiral)
        rev_smiles = constructMol(frags, adj, asMol= False, useChiral= hasChiral)
        if rev_smiles is None:
            raise ValueError()

        if hasChiral & (ecfp is not None):
            rev_smiles = isomer_search(rev_smiles, ecfp)

        if Chem.CanonSmiles(smiles, useChiral= int(hasChiral)) != Chem.CanonSmiles(rev_smiles, useChiral= int(hasChiral)):
            if Chem.CanonSmiles(smiles, useChiral= 0) == Chem.CanonSmiles(rev_smiles, useChiral= 0):
                correct = 2
            else:
                correct = 0
                if verbose: print(f'[UNMATCH] {smiles}, {rev_smiles}', flush= True)
        else:
            correct = 3 if hasChiral else 1
    except:
        rev_smiles = None
        correct = 0
        if verbose: print(f'[ERROR] {smiles}', flush= True)

    return rev_smiles, correct


def constructMol(frag_smiles: list, adj: list, asMol: bool= False, useChiral: bool= True):
    """
        frag_smiles: list of fragments as SMILES,
        adj: torch.Tensor or list shape= (len(frag_smiles), len(frag_smiles)), 0: none, 1: single, 2: double, 3: triple
        validity correction: choose largest fragments and check Valence
    """
    # process options
    blg = rdBase.BlockLogs()
    lg = RDLogger.logger()
    lg.setLevel(RDLogger.CRITICAL)
    # stand = Standardizer()
    remover = Chem.RemoveHsParameters()
    remover.removeDegreeZero = False

    # one node
    if len(adj) < 2:
        mol = AllChem.ReplaceSubstructs(Chem.MolFromSmiles(frag_smiles[0]), Chem.MolFromSmiles('*'), Chem.MolFromSmiles('[H]'), replaceAll= True)[0]
        mol = Chem.RemoveHs(mol, remover, sanitize= False)
        # mol = stand.standardize(mol)
        smi = Chem.MolToSmiles(mol)
        try:
            if Chem.SanitizeMol(mol, catchErrors= True) == Chem.rdmolops.SanitizeFlags.SANITIZE_KEKULIZE:
                # print(smi, flush= True)
                smi = smi.replace('[n+]', '[n]')
            smi = Chem.CanonSmiles(smi)
            if asMol: smi = Chem.MolFromSmiles(smi)
        except:
            smi = None
        return smi
    else:
        adj = np.array(adj)

    # smiles to mol
    fragments = [Chem.MolFromSmiles(s) for s in frag_smiles]

    # assign AtomMapNum and combine fragments
    for i, frag in enumerate(fragments):
        n = delta = 0.25
        for atom in frag.GetAtoms():
            if atom.GetAtomicNum() == 0:
                if atom.GetAtomMapNum() == 0:
                    assert n < 10
                    atom.SetDoubleProp('dummyMapNumber', n + 10*i)
                    n += delta
                else:
                    atom.SetDoubleProp('dummyMapNumber', delta * atom.GetAtomMapNum() + 10*i)
        if i == 0:
            combo = frag
        else:
            combo = Chem.CombineMols(combo, frag)
    rwcombo = RWMol(combo)

    tril = np.tril(adj)
    indices = list(map(list, np.where(tril)))
    bond_types = tril[tril!=0].tolist()
    stereos = [list(bond.GetStereoAtoms()) for bond in rwcombo.GetBonds() if bond.GetStereo() != Chem.rdchem.BondStereo.STEREONONE]
    stereo_atoms = list(chain.from_iterable(stereos))

    # add Bond between fragments
    for i1, i2, b in zip(*indices, bond_types):
        start_end = [i1, i2]
        connect_idxs = []
        dummy_idxs = []
        bonds = []
        for idx in start_end:
            for i, a in enumerate(rwcombo.GetAtoms()):
                if a.HasProp('dummyMapNumber'):
                    mapnum = a.GetDoubleProp('dummyMapNumber')
                    if mapnum == 10*idx + delta:
                        a.ClearProp('dummyMapNumber')
                        if len(a.GetNeighbors()) == 1:
                            neighbor = a.GetNeighbors()[0]
                            dummy_idxs.append(a.GetIdx())
                            connect_idxs.append(neighbor.GetIdx())
                            bonds.append(int(rwcombo.GetBondBetweenAtoms(a.GetIdx(), neighbor.GetIdx()).GetBondTypeAsDouble()))
                    elif (mapnum > 10*idx+delta) & (mapnum < 10*(idx+1)):
                        a.SetDoubleProp('dummyMapNumber', mapnum - delta)
                else:
                    continue

        if len(connect_idxs) == 2:
            c1, c2 = connect_idxs
            b = min(bonds)
            rwcombo.AddBond(c1, c2, BONDTYPES[b])
            for c, d in zip(connect_idxs, dummy_idxs):
                rwcombo.RemoveBond(c, d)
            if (dummy_idxs[0] in stereo_atoms) | (dummy_idxs[1] in stereo_atoms):
                idxs = {dummy_idxs[0]: c2, dummy_idxs[1]: c1}
                for bn, aids in enumerate(stereos):
                    stereos[bn] = [idxs[a] if a in idxs.keys() else a for a in aids]

    # assign cis/trans stereo
    i = 0
    for bond in rwcombo.GetBonds():
        if bond.GetStereo() != Chem.rdchem.BondStereo.STEREONONE:
            bond.SetStereoAtoms(*stereos[i])
            i += 1

    # remove dummy atoms
    try:
        mol = AllChem.ReplaceSubstructs(rwcombo, Chem.MolFromSmiles('*'), Chem.MolFromSmiles('[H]'), replaceAll= True)[0]
        mol = Chem.RemoveHs(mol, remover, sanitize= False)
        mol = AllChem.ReplaceSubstructs(mol, Chem.MolFromSmiles('[H]'), Chem.MolFromSmiles('*'), replaceAll= True)[0]
        mol = AllChem.DeleteSubstructs(mol, Chem.MolFromSmiles('*'))
        for _ in range(len(fragments)):
            if Chem.SanitizeMol(mol, catchErrors= True) == Chem.rdmolops.SanitizeFlags.SANITIZE_NONE:
                break

        mol.UpdatePropertyCache(strict=True)
        if useChiral:
            chirals = Chem.FindMolChiralCenters(mol)
            if chirals:
                tmpmol = Chem.MolFromSmiles(Chem.MolToSmiles(mol, canonical= True))
                try:
                    tmpmol = Chem.RenumberAtoms(tmpmol, tmpmol.GetSubstructMatch(mol))
                    for (a1, c1), (a2, c2) in zip(chirals, Chem.FindMolChiralCenters(tmpmol)):
                        if (a1 == a2) & (c1 != c2):
                            mol.GetAtomWithIdx(a1).InvertChirality()
                except Exception as e:
                    pass
        # mol = stand.standardize(mol)    # sanitize
        smi = Chem.MolToSmiles(mol, canonical= True)
        tmpmol = deepcopy(mol)
        err = Chem.SanitizeMol(tmpmol, catchErrors= True)
        if err == Chem.rdmolops.SanitizeFlags.SANITIZE_KEKULIZE:
            smi = smi.replace('[n+]', '[n]')
        smi = Chem.CanonSmiles(smi)
    except Exception as e:
        print(str(e), flush= True)
        return None

    if asMol:
        return Chem.MolFromSmiles(smi)
    else:
        return smi


def constructMolwithECFP(frag_smiles: list, adj: list, ecfp: list, radius: int):
    smi = constructMol(frag_smiles, adj, asMol= False)

    return isomer_search(smi, ecfp, radius)


def constructMolwithTimeout(frag_smiles: list, adj: list, asMol: bool= False, useChiral: bool= True, timeout: int= 300):
    @timeout_decorator.timeout(timeout, use_signals= False)
    def construct(frag_smiles: list, adj: list, asMol: bool= False, useChiral: bool= True):
        return constructMol(frag_smiles, adj, asMol, useChiral)
    try:
        smi = construct(frag_smiles, adj, asMol, useChiral)
    except Exception as e:
        print(e, flush= True)
        smi = None
    return smi



## scripts/utils/data.py

In [ ]:
import torch
from torch.utils.data import Dataset
from torch.nn.utils.rnn import pad_sequence


class ListDataset(Dataset):
    def __init__(self, frag_indices: list , positions: list, prop: torch.Tensor) -> None:
        """
        frag_indices, positions: list of torch.Tensors with different lengths
        ecfps, prop: torch.Tensor
        """
        super().__init__()
        self.ecfps = None
        self.frag_indices = frag_indices
        self.positions = positions
        self.prop = prop

    def __len__(self):
        return len(self.frag_indices)

    def __getitem__(self, index) -> [torch.Tensor, torch.Tensor, torch.Tensor]:
        if self.ecfps is None:
            return self.frag_indices[index], self.positions[index], self.prop[index]
        else:
            return self.ecfps[index], self.frag_indices[index], self.positions[index], self.prop[index]

    def set_stereo(self, ecfps):
        self.ecfps = ecfps


def collate_pad_fn(batch):
    frag_indices, positions, props = zip(*batch)
    frag_indices = pad_sequence(frag_indices, batch_first= True, padding_value= 0)
    positions = pad_sequence(positions, batch_first= True, padding_value= 0)
    props = torch.stack(props)

    return frag_indices, positions, props

def collate_stereo_fn(batch):
    ecfps, frag_indices, positions, props = zip(*batch)
    ecfps = torch.stack(ecfps)
    frag_indices = pad_sequence(frag_indices, batch_first= True, padding_value= 0)
    positions = pad_sequence(positions, batch_first= True, padding_value= 0)
    props = torch.stack(props)

    return ecfps, frag_indices, positions, props

## scripts/utils/fragmentation.py

In [ ]:
import os
from itertools import chain
from rdkit import Chem
from rdkit.Chem import RWMol, BRICS

# from utils.medchemfrag import decomposition


def FindBRICS(mol, bonds: list= None, AtomDone: set= None):
    bonds = bonds if bonds is not None else []
    AtomDone = AtomDone if AtomDone is not None else set([])

    for idxs, _ in BRICS.FindBRICSBonds(mol):
        idxs = sorted(idxs)
        if (idxs in bonds) or (len(set(idxs) & AtomDone) > 1):  # both atoms have already been searched.
            continue
        else:
            bonds.append(sorted(idxs))
            AtomDone = AtomDone | set(idxs)
    return bonds, AtomDone


def FindRings(mol, bonds: list= None, AtomDone: set= None):
    bonds = bonds if bonds is not None else []
    AtomDone = AtomDone if AtomDone is not None else set([])
    for bond in mol.GetBonds():
        begin = bond.GetBeginAtom()
        end = bond.GetEndAtom()

        if (begin.GetIdx() in AtomDone) & (end.GetIdx() in AtomDone):
            continue

        # bond between rings, ring and C-ANY
        if (begin.IsInRing() | end.IsInRing()) & (not bond.IsInRing()) & (bond.GetBondTypeAsDouble() < 2):
            if begin.IsInRing() & end.IsInRing():
                neighbor = 1
            elif begin.IsInRing():
                neighbor = len(end.GetNeighbors()) - 1
            elif end.IsInRing():
                neighbor = len(begin.GetNeighbors()) - 1
            else:
                neighbor = 0

            if neighbor > 0:
                idxs = sorted([begin.GetIdx(), end.GetIdx()])
                if idxs not in bonds:
                    bonds.append(idxs)
                    AtomDone = AtomDone | set(idxs)
    return bonds, AtomDone


# def FindStereo(mol, bonds: list= None):
#     """
#     find cis/trans stereo type
#     """
#     bonds = bonds if bonds is not None else []
#     for bond in mol.GetBonds():
#         if bond.GetStereo() != Chem.rdchem.BondStereo.STEREONONE:
#             stereo_atoms = list(bond.GetStereoAtoms())
#             atom_idxs = [bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()]
#             for aid in atom_idxs:
#                 neighbors = mol.GetAtomWithIdx(aid).GetNeighbors()
#                 if len(neighbors) > 2:
#                     idx = [n.GetIdx() for n in neighbors if (n.GetIdx() not in stereo_atoms) & (n.GetIdx() not in atom_idxs)][0]
#                     idxs = sorted([aid, idx])
#                     if idxs not in bonds:
#                         bonds.append(idxs)
#     return bonds

def find_BRICSbonds(mol) -> list:
    return sorted([sorted(idxs) for idxs, _ in BRICS.FindBRICSBonds(mol)], key= lambda idxs: idxs[1])

def find_rings(mol) -> list:
    return sorted(FindRings(mol)[0], key= lambda idxs: idxs[1])

def find_MedChemFrag(mol) -> list:
    return sorted([sorted(idxs) for idxs in decomposition(mol)], key= lambda idxs: idxs[1])

def find_BRICSbonds_and_rings(mol) -> list:
    """
    Find bonds which are BRICS bonds or single bonds between rings.
    """
    # bonds, AtomDone = FindBRICS(mol)
    bonds = find_BRICSbonds(mol)
    return sorted(FindRings(mol, bonds= bonds)[0], key= lambda idxs: idxs[1])

## scripts/utils/decompose.py

In [ ]:
from copy import deepcopy
import numpy as np
from itertools import chain

from rdkit import Chem, RDLogger
from rdkit.Chem import RWMol, AllChem

# from utils.fragmentation import find_BRICSbonds, find_BRICSbonds_and_rings, find_MedChemFrag

lg = RDLogger.logger()
lg.setLevel(RDLogger.CRITICAL)

BONDTYPES = {1: Chem.rdchem.BondType.SINGLE,
             2: Chem.rdchem.BondType.DOUBLE,
             3: Chem.rdchem.BondType.TRIPLE}


def setAtomMapNumsWithIdxs(mol, indexs= None):
    indexs = indexs if indexs else range(mol.GetNumAtoms())
    for i, atom in zip(indexs, mol.GetAtoms()):
        atom.SetAtomMapNum(i)

def clearAtomMapNums(mol):
    for atom in mol.GetAtoms():
        atom.SetAtomMapNum(0)

def check_fragSize(mol, minSize: int= 1, maxDegree: int= 32):
    frags = Chem.GetMolFrags(mol, asMols= True, sanitizeFrags= False)
    cut = True
    for f in frags:
        Chem.SanitizeMol(f, catchErrors= True)
        f = Chem.MolFromSmiles(Chem.MolToSmiles(f))
        # check atom num exclude dummy atom
        dummynum = sum([a.GetAtomicNum() == 0 for a in f.GetAtoms()])
        atomnum = f.GetNumAtoms() - dummynum
        if (atomnum < minSize) | (dummynum > maxDegree):
            cut = False
            break

    return cut

def HydrogenMatch(f, f_dash, uniquify: bool= True):
    orders = f_dash.GetSubstructMatches(f, uniquify= uniquify)
    canonOrder = None
    for order in orders:
        f_ordered = Chem.RenumberAtoms(f_dash, order)
        assert f.GetNumAtoms() == f_ordered.GetNumAtoms()
        for a, a_dash in zip(f.GetAtoms(), f_ordered.GetAtoms()):
            if a.GetAtomicNum() != a_dash.GetAtomicNum(): break
            elif a.GetNumExplicitHs() != a_dash.GetNumExplicitHs(): break
        else:
            canonOrder = order
            break

    if (canonOrder is None) & uniquify:
        canonOrder = HydrogenMatch(f, f_dash, uniquify= False)

    if (canonOrder is None) & bool(orders):
        canonOrder = orders[0]

    return canonOrder

def MapNumsToAdj(bondMapNums: list, bond_types: list):
    n_frags = len(bondMapNums)
    if n_frags == 1:
        adj = [[0]]
    else:
        n_bonds = len(bond_types)
        adj = [[0] * n_frags for _ in range(len(bondMapNums))]

        for b in range(1, n_bonds+1):
            i1, i2 = [i for i in range(n_frags) if b in bondMapNums[i]]
            adj[i1][i2] = bond_types[b-1]
            adj[i2][i1] = bond_types[b-1]

    return adj


def MolToBRICSfragments(mol, minFragSize: int= 1, maxDegree: int= 32, useChiral: bool= True, useStereo: bool= False):
    rwmol = RWMol(mol)
    numatoms = rwmol.GetNumAtoms()

    # search break bonds
    # matches = find_BRICSbonds(rwmol)
    matches = find_BRICSbonds_and_rings(rwmol)
    # matches = find_MedChemFrag(mol)

    # decompose mol
    bond_types = []
    chiral_centers = []
    stereos = [list(bond.GetStereoAtoms()) for bond in rwmol.GetBonds() if bond.GetStereo() != Chem.rdchem.BondStereo.STEREONONE]
    stereo_atoms = list(chain.from_iterable(stereos))
    i = 0
    while i < len(matches):
        match = matches[i]
        a1, a2 = match[0], match[1]
        bond = rwmol.GetBondBetweenAtoms(a1, a2)
        bond_type = int(bond.GetBondTypeAsDouble())

        # stereo chem
        if useStereo & (bond.GetBondType() == BONDTYPES[2]):
            matches.remove(match)
            continue

        # check whether fragment size is bigger than min-size
        tmpmol = deepcopy(rwmol)
        rwmol.RemoveBond(a1, a2)
        for a in match:
            rwmol.AddAtom(Chem.Atom(0))
            rwmol.GetAtomWithIdx(numatoms).SetAtomMapNum(i+1)
            # rwmol.AddBond(a, numatoms, BONDTYPES[1])
            rwmol.AddBond(a, numatoms, BONDTYPES[bond_type])

            numatoms += 1

        if check_fragSize(rwmol.GetMol(), minSize= minFragSize, maxDegree= maxDegree):
            i += 1
            bond_types.append(bond_type)
            chiral_centers += [a for a in match if tmpmol.GetAtomWithIdx(a).HasProp('_CIPCode')]
            # fix cis/trans stereo
            if (a1 in stereo_atoms) | (a2 in stereo_atoms):
                idxs = {a1: numatoms-1, a2: numatoms-2}
                for bn, aids in enumerate(stereos):
                    stereos[bn] = [idxs[a] if a in idxs.keys() else a for a in aids]
        else:
            rwmol = deepcopy(tmpmol)      # role back
            numatoms -= len(match)
            matches.remove(match)

    # assign cis/trans stereo
    i = 0
    for bond in rwmol.GetBonds():
        if bond.GetStereo() != Chem.rdchem.BondStereo.STEREONONE:
            try:
                bond.SetStereoAtoms(*stereos[i])
            except:
                pass
            i += 1

    # fragmentation
    fragments_numed = Chem.GetMolFrags(rwmol, asMols= True, sanitizeFrags= False)
    fragments = [Chem.MolFromSmiles(Chem.MolToSmiles(f, canonical= True)) for f in Chem.GetMolFrags(rwmol, asMols= True, sanitizeFrags= False)]

    # only one fragment
    if len(fragments) == 1:
        return [Chem.MolToSmiles(mol)], [0], [[0]]

    # convert to canonical atom orders
    bondMapNums = []
    etc = 1
    for f_numed, f in zip(fragments_numed, fragments):
        Chem.SanitizeMol(f_numed, catchErrors= True)

        # make connect map
        bondMapNum = sorted([a.GetAtomMapNum() for a in f.GetAtoms() if a.GetAtomMapNum()])
        if len(bondMapNum) == 0:
            bondMapNum = [len(matches)+etc]
            etc += 1
        bondMapNums.append(bondMapNum)

        # reset chirality
        if useChiral:
            canonOrderIdxs = HydrogenMatch(f, f_numed)
            f_renumed = Chem.RenumberAtoms(f_numed, canonOrderIdxs)
            chirals_f = Chem.FindMolChiralCenters(f)
            chirals_f_renumed = Chem.FindMolChiralCenters(f_renumed)
            if len(chirals_f) != len(chirals_f_renumed):
                chirals_nums, _ = zip(*chirals_f) if chirals_f else ([], [])
                for a, ctype in chirals_f_renumed:
                    if a not in chirals_nums:
                        f.GetAtomWithIdx(a).SetProp('_CIPCode', ctype)
                        f.GetAtomWithIdx(a).SetChiralTag(f_renumed.GetAtomWithIdx(a).GetChiralTag())
                Chem.AssignCIPLabels(f)
                chirals_f = Chem.FindMolChiralCenters(f)
            for (a1, c1), (a2, c2) in zip(chirals_f, chirals_f_renumed):
                if (a1 == a2) & (c1 != c2):
                    f.GetAtomWithIdx(a1).InvertChirality()

            # assign cis/trans stereo
            for bond in f_renumed.GetBonds():
                if bond.GetStereo() != Chem.rdchem.BondStereo.STEREONONE:
                    a1, a2 = bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()
                    try:
                        f.GetBondBetweenAtoms(a1, a2).SetStereo(bond.GetStereo())
                        f.GetBondBetweenAtoms(a1, a2).SetStereoAtoms(*list(bond.GetStereoAtoms()))
                    except:
                        pass

        # numbering dummy atoms
        dummys = [(a.GetAtomMapNum(), a.GetIdx()) for a in f.GetAtoms() if a.GetAtomMapNum()]
        sort_dummys = sorted(dummys, key= lambda dummys: dummys[0])
        for n, (_, i) in enumerate(sort_dummys):
            f.GetAtomWithIdx(i).SetAtomMapNum(n+1)

    frag_smiles = []
    for f in fragments:
        # Chem.SanitizeMol(f)   # miss stereo chemistry
        frag_smiles.append(Chem.MolToSmiles(f).replace('\\\\', '\\'))

    return frag_smiles, bond_types, bondMapNums


def MolFromFragments(frag_smiles: list, adj: list, asMol: bool= False, useChiral: bool= True):
    """
    frag_smiles: list of fragments as SMILES,
    adj: torch.Tensor or list shape= (len(frag_smiles), len(frag_smiles)), 0: none, 1: single, 2: double, 3: triple
    """
    if len(frag_smiles) == 1:
        mol = Chem.MolFromSmiles(Chem.CanonSmiles(frag_smiles[0])) if asMol else Chem.CanonSmiles(frag_smiles[0])
        return mol

    # remove padding
    adj = np.array(adj)

    # smiles to mol
    fragments = [Chem.MolFromSmiles(s) for s in frag_smiles]

    # assign AtomMapNum and combine fragments
    for i, frag in enumerate(fragments):
        n = delta = 0.25
        for atom in frag.GetAtoms():
            if atom.GetAtomicNum() == 0:
                if atom.GetAtomMapNum() == 0:
                    assert n < 10
                    atom.SetDoubleProp('dummyMapNumber', n + 10*i)
                    n += delta
                else:
                    atom.SetDoubleProp('dummyMapNumber', delta * atom.GetAtomMapNum() + 10*i)

        if i == 0:
            combo = frag
        else:
            combo = Chem.CombineMols(combo, frag)
    rwcombo = RWMol(combo)

    tril = np.tril(adj)
    indices = list(map(list, np.where(tril)))
    bond_types = tril[tril!=0].tolist()
    stereos = [list(bond.GetStereoAtoms()) for bond in rwcombo.GetBonds() if bond.GetStereo() != Chem.rdchem.BondStereo.STEREONONE]
    stereo_atoms = list(chain.from_iterable(stereos))

    # add Bond between fragments
    for i1, i2, b in zip(*indices, bond_types):
        start_end = [i1, i2]
        connect_idxs = []
        dummy_idxs = []
        bonds = []
        for idx in start_end:
            for i, a in enumerate(rwcombo.GetAtoms()):
                if a.HasProp('dummyMapNumber'):
                    mapnum = a.GetDoubleProp('dummyMapNumber')
                    if mapnum == 10*idx + delta:
                        a.ClearProp('dummyMapNumber')
                        assert len(a.GetNeighbors()) == 1
                        neighbor = a.GetNeighbors()[0]
                        dummy_idxs.append(a.GetIdx())
                        connect_idxs.append(neighbor.GetIdx())
                        bonds.append(int(rwcombo.GetBondBetweenAtoms(a.GetIdx(), neighbor.GetIdx()).GetBondTypeAsDouble()))
                    elif (mapnum > 10*idx+delta) & (mapnum < 10*(idx+1)):
                        a.SetDoubleProp('dummyMapNumber', mapnum - delta)
                else:
                    continue

        assert len(connect_idxs) == 2
        c1, c2 = connect_idxs
        b = min(bonds)
        rwcombo.AddBond(c1, c2, BONDTYPES[b])
        for c, d in zip(connect_idxs, dummy_idxs):
            rwcombo.RemoveBond(c, d)
        if (dummy_idxs[0] in stereo_atoms) | (dummy_idxs[1] in stereo_atoms):
            idxs = {dummy_idxs[0]: c2, dummy_idxs[1]: c1}
            for bn, aids in enumerate(stereos):
                stereos[bn] = [idxs[a] if a in idxs.keys() else a for a in aids]

    # assign cis/trans stereo
    i = 0
    for bond in rwcombo.GetBonds():
        if bond.GetStereo() != Chem.rdchem.BondStereo.STEREONONE:
            bond.SetStereoAtoms(*stereos[i])
            i += 1

    # remove dummy atoms
    remover = Chem.RemoveHsParameters()
    remover.removeDegreeZero = False
    mol = AllChem.ReplaceSubstructs(rwcombo, Chem.MolFromSmiles('*'), Chem.MolFromSmiles('[H]'), replaceAll= True)[0]
    mol = Chem.RemoveHs(mol, remover, sanitize= False)
    mol = AllChem.ReplaceSubstructs(mol, Chem.MolFromSmiles('[H]'), Chem.MolFromSmiles('*'), replaceAll= True)[0]
    mol = AllChem.DeleteSubstructs(mol, Chem.MolFromSmiles('*'))

    # sanitize fragments
    for _ in range(len(fragments)):
        if Chem.SanitizeMol(mol, catchErrors= True) == Chem.rdmolops.SanitizeFlags.SANITIZE_NONE:
            break

    # check chirality
    if useChiral:
        chirals = Chem.FindMolChiralCenters(mol)
        if chirals:
            tmpmol = Chem.MolFromSmiles(Chem.MolToSmiles(mol, canonical= True))
            tmpmol = Chem.RenumberAtoms(tmpmol, tmpmol.GetSubstructMatch(mol))
            for (a1, c1), (a2, c2) in zip(chirals, Chem.FindMolChiralCenters(tmpmol)):
                if (a1 == a2) & (c1 != c2):
                    mol.GetAtomWithIdx(a1).InvertChirality()

    # mol = stand.standardize(mol)
    # Chem.SanitizeMol(mol)
    smi = Chem.MolToSmiles(mol, canonical= True)

    if asMol:
        return Chem.MolFromSmiles(smi)
    else:
        return smi


if __name__ == '__main__':
    smi = 'COc1ccccc1/C=C/C=C(\C#N)C(=O)Nc1ccc(C(=O)N(C)C)cc1'
    frag_smiles, bond_types, bondMapNums = MolToBRICSfragments(Chem.MolFromSmiles(smi), useStereo= True)
    # n_frags = len(frag_smiles)
    # tmp_ecfps = [[0, 0] for _ in range(len(frag_smiles))]
    # tree = make_DFStree(list(range(n_frags)), tmp_ecfps, bond_types, bondMapNums)
    # fids = tree.dgl_graph.ndata['fid'].squeeze(-1)
    # adj = tree.adjacency_matrix().to_dense()
    # frag_smiles = [frag_smiles[fid] for fid in fids]
    adj = MapNumsToAdj(bondMapNums, bond_types)
    smi_rev = MolFromFragments(frag_smiles, adj)
    print(smi)
    print(smi_rev)

COc1ccccc1/C=C/C=C(\C#N)C(=O)Nc1ccc(C(=O)N(C)C)cc1
COc1ccccc1C=C/C=C(\C#N)C(=O)Nc1ccc(C(=O)N(C)C)cc1


## scripts/utils/fragmentation.py

In [ ]:
import os
from itertools import chain
from rdkit import Chem
from rdkit.Chem import RWMol, BRICS

# from utils.medchemfrag import decomposition


def FindBRICS(mol, bonds: list= None, AtomDone: set= None):
    bonds = bonds if bonds is not None else []
    AtomDone = AtomDone if AtomDone is not None else set([])

    for idxs, _ in BRICS.FindBRICSBonds(mol):
        idxs = sorted(idxs)
        if (idxs in bonds) or (len(set(idxs) & AtomDone) > 1):  # both atoms have already been searched.
            continue
        else:
            bonds.append(sorted(idxs))
            AtomDone = AtomDone | set(idxs)
    return bonds, AtomDone


def FindRings(mol, bonds: list= None, AtomDone: set= None):
    bonds = bonds if bonds is not None else []
    AtomDone = AtomDone if AtomDone is not None else set([])
    for bond in mol.GetBonds():
        begin = bond.GetBeginAtom()
        end = bond.GetEndAtom()

        if (begin.GetIdx() in AtomDone) & (end.GetIdx() in AtomDone):
            continue

        # bond between rings, ring and C-ANY
        if (begin.IsInRing() | end.IsInRing()) & (not bond.IsInRing()) & (bond.GetBondTypeAsDouble() < 2):
            if begin.IsInRing() & end.IsInRing():
                neighbor = 1
            elif begin.IsInRing():
                neighbor = len(end.GetNeighbors()) - 1
            elif end.IsInRing():
                neighbor = len(begin.GetNeighbors()) - 1
            else:
                neighbor = 0

            if neighbor > 0:
                idxs = sorted([begin.GetIdx(), end.GetIdx()])
                if idxs not in bonds:
                    bonds.append(idxs)
                    AtomDone = AtomDone | set(idxs)
    return bonds, AtomDone


# def FindStereo(mol, bonds: list= None):
#     """
#     find cis/trans stereo type
#     """
#     bonds = bonds if bonds is not None else []
#     for bond in mol.GetBonds():
#         if bond.GetStereo() != Chem.rdchem.BondStereo.STEREONONE:
#             stereo_atoms = list(bond.GetStereoAtoms())
#             atom_idxs = [bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()]
#             for aid in atom_idxs:
#                 neighbors = mol.GetAtomWithIdx(aid).GetNeighbors()
#                 if len(neighbors) > 2:
#                     idx = [n.GetIdx() for n in neighbors if (n.GetIdx() not in stereo_atoms) & (n.GetIdx() not in atom_idxs)][0]
#                     idxs = sorted([aid, idx])
#                     if idxs not in bonds:
#                         bonds.append(idxs)
#     return bonds

def find_BRICSbonds(mol) -> list:
    return sorted([sorted(idxs) for idxs, _ in BRICS.FindBRICSBonds(mol)], key= lambda idxs: idxs[1])

def find_rings(mol) -> list:
    return sorted(FindRings(mol)[0], key= lambda idxs: idxs[1])

def find_MedChemFrag(mol) -> list:
    return sorted([sorted(idxs) for idxs in decomposition(mol)], key= lambda idxs: idxs[1])

def find_BRICSbonds_and_rings(mol) -> list:
    """
    Find bonds which are BRICS bonds or single bonds between rings.
    """
    # bonds, AtomDone = FindBRICS(mol)
    bonds = find_BRICSbonds(mol)
    return sorted(FindRings(mol, bonds= bonds)[0], key= lambda idxs: idxs[1])

## scripts/utils/mask.py

In [ ]:
import torch

# Reference from https://github.com/pytorch/tutorials/blob/main/beginner_source/translation_transformer.py
def generate_square_subsequent_mask(length: int, device: torch.device= 'cpu'):
    mask = (torch.triu(torch.ones((length, length), device= device)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask

def create_mask(src: torch.Tensor, tgt: torch.Tensor, pad_idx: int= 0, batch_first: bool= True):
    device = src.device
    src_seq_len = src.shape[1] if batch_first else src.shape[0]
    tgt_seq_len = tgt.shape[1] if batch_first else tgt.shape[0]

    tgt_mask = generate_square_subsequent_mask(tgt_seq_len, device)
    src_mask = torch.zeros((src_seq_len, src_seq_len), device= device).type(torch.bool)

    src_padding_mask = src == pad_idx if batch_first else (src == pad_idx).transpose(0, 1)
    tgt_padding_mask = tgt == pad_idx if batch_first else (tgt == pad_idx).transpose(0, 1)
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask


## scripts/utils/medchemfrag.py

In [ ]:
from rdkit import Chem
from rdkit.Chem import Descriptors, AllChem

"""
Reference: http://molmodel.com/hg/medchem_fragment_splitter (2023/02/13)
"""

rules = [
    "[R]-!@[$([CX4;H2,H1,H0])]",  # 1
    "[a]-!@[$([NX3;H1,H0]),$([OX2;H0]),$([SX2;H0])]-!@[$([C;H2,H1,H0]);!$([CX3]=[OX1])]",  # 2
    "[a]-!@[$([NX3;H1,H0]),$([OX2;H0]),$([SX2;H0])]-!@[a]",  # 3
    "[a]-!@[$([CX3]=[OX1,NX2,SX1,CX3])]-!@[$([CX4;H2,H1,H0])]",  # 4
    "[c]-!@[$([CX3]=[OX1,NX2,SX1,$([CX3;H2])])]-!@[c]",  # 5.1
    "[n]-!@[$([CX3]=[OX1,NX2,SX1,$([CX3;H2])])]-!@[c]",  # 5.2
    "[$([CX4;H2,H1,H0])]-!@[CX3](=[OX1])[OX2;H0]",  # 6
    "[$([CX4;H2,H1,H0])]-!@[OX2;H0][CX3](=[OX1])",  # 7
    "[a]-!@[CX3](=[OX1])O-!@[$([CX4;H2,H1,H0])]",  # 8
    "[a]-!@[CX3](=[OX1])O-!@[a]",  # 9
    "[a]-!@[NX2;H0]=[NX2;H0]-!@[$([CX4;H2,H1,H0])]",  # 10
    "[a]-!@[NX2;H0]=[NX2;H0]-!@[a]",  # 11
    "[a]-!@[NX3;H1]-!@[$([CX3;H0](=[OX1]))]-!@[$([CX4;H2,H1,H0])]",  # 12
    "[a]-!@[$([CX3;H0](=[OX1]))]-!@[NX3;H1]-!@[$([CX4;H2,H1,H0])]",  # 13
    "[a]-!@[NX3;H1]-!@[$([CX3;H0](=[OX1]))]-!@[a]",  # 14
    "[a]-!@[$([CX3;H0](=[OX1]))]-!@[NX3;H1]-!@[a]",  # 15
    "[a]-!@[SX4](=[OX1])(=[OX1])[NX3;H1]-!@[$([CX4;H2,H1,H0])]",  # 16
    "[a]-!@[NX3;H1][SX4](=[OX1])(=[OX1])-!@[$([CX4;H2,H1,H0])]",  # 17
    "[a]-!@[SX4](=[OX1])(=[OX1])[NX3;H1]-!@[a]",  # 18
    "[a]-!@[NX3;H1][SX4](=[OX1])(=[OX1])-!@[NX3;H1]-!@[a]",  # 19
    "[$([CX4;H2,H1,H0])]-!@[NX3][CX3](=[OX1])",  # 20
    "[$([CX4;H2,H1,H0])]-!@[CX3](=[OX1])[NX3]",  # 21
    "[$([CX4;H2,H1,H0])]-!@[$([NX3;H1,H0])]",  # 22
    "[$([CX4;H2,H1,H0])]-!@[$([OX2;H0])]",  # 23
    "[$([CX4;H2,H1,H0])]-!@[$([SX2;H0])]",  # 24
    "[$([CX4;H2,H1,H0])]-!@[SX4](=[OX1])(=[OX1])[NX3;H1]",  # 25
    "[$([CX4;H2,H1,H0])]-!@[NX3;H1][SX4](=[OX1])(=[OX1])"  # 26
]

index_all = [
    [(0, 1)],  # 1
    [(1, 2)],  # 2
    [(0, 1), (1, 2)],  # 3
    [(1, 2)],  # 4
    [(0, 1), (1, 2)],  # 5.1
    [(1, 2)],  # 5.2
    [(0, 1)],  # 6
    [(0, 1)],  # 7
    [(3, 4)],  # 8
    [(0, 1), (3, 4)],  # 9
    [(2, 3)],  # 10
    [(0, 1), (2, 3)],  # 11
    [(2, 3)],  # 12
    [(2, 3)],  # 13
    [(0, 1), (2, 3)],  # 14
    [(0, 1), (2, 3)],  # 15
    [(4, 5)],  # 16
    [(4, 5)],  # 17
    [(0, 1), (4, 5)],  # 18
    [(0, 1), (4, 5)],  # 19
    [(0, 1)],  # 20
    [(0, 1)],  # 21
    [(0, 1)],  # 22
    [(0, 1)],  # 23
    [(0, 1)],  # 24
    [(0, 1)],  # 25
    [(0, 1)],  # 26
]

def decomposition(molecule, smarts: list= None):
    only_smarts = (smarts is None)

    atomPairs = []
    if not only_smarts:
        for index in range(len(smarts)):
            smartFragment = Chem.MolFromSmiles(smarts[index])
            atomPairsLoc = molecule.GetSubstructMatches(smartFragment)
            for atomPair in atomPairsLoc:
                atomPairs.append(atomPair)
    else:
        for index in range(len(rules)):
            atomPairsLoc = molecule.GetSubstructMatches(Chem.MolFromSmarts(rules[index]))
            for nb in range(len(index_all[index])):
                index1 = index_all[index][nb][0]
                index2 = index_all[index][nb][1]
                for atomPair in atomPairsLoc:
                    atomIndex1 = atomPair[index1]
                    atomIndex2 = atomPair[index2]
                    atomPairs.append((atomIndex1, atomIndex2))

    if (len(atomPairs) == 0):
        return []

    atomPairs = list(set(atomPairs))
    bonds = list()
    if not only_smarts:
        flag = False
        for atomIndexes1 in atomPairs:
            for atomIndexes2 in atomPairs:
                if atomIndexes1 == atomIndexes2:
                    continue
                else:
                    for a1 in atomIndexes1:
                        for a2 in atomIndexes2:
                            if a1 == a2:
                                flag = True
                                if len(atomIndexes1) > len(atomIndexes2):
                                    if atomIndexes2 in atomPairs:
                                        atomPairs.remove(atomIndexes2)
                                else:
                                    if atomIndexes1 in atomPairs:
                                        atomPairs.remove(atomIndexes1)
                                    break
                        if flag:
                            break
                flag = False

    for atomIndexes in atomPairs:
        for a1 in atomIndexes:
            for a2 in atomIndexes:
                if a1 == a2:
                    continue
                else:
                    bond = molecule.GetBondBetweenAtoms(a1, a2)
                    if (bond != None):
                        idxs = sorted([a1, a2])
                        if idxs not in bonds:
                            bonds.append(idxs)
    # bonds = list(set(bonds))
    return bonds


def add_nitrogen_charges(m):
    m.UpdatePropertyCache(strict=False)
    ps = Chem.DetectChemistryProblems(m)
    if not ps:
        Chem.SanitizeMol(m)
        return m
    for p in ps:
        if p.GetType()=='AtomValenceException':
            at = m.GetAtomWithIdx(p.GetAtomIdx())
            if at.GetAtomicNum()==7 and at.GetFormalCharge()==0 and at.GetExplicitValence()==4:
                at.SetFormalCharge(1)
    Chem.SanitizeMol(m)
    return m

## scripts/utils/metrics.py

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

def batched_kl_divergence(mu: torch.Tensor, ln_var: torch.Tensor):
        return (- 0.5 * torch.sum(1 + ln_var - mu.pow(2) - ln_var.exp(), dim= -1)).mean(dim= 0)

def cosine_matrix(a: torch.Tensor, b: torch.Tensor, p: int= 2, dim: int= 1):
    """
    a.shape = (*, dim)
    b.shape = (*, dim)
    a.dim == b.dim

    return matrix.shape= (a.shape[0], b.shape[0])
    """
    return torch.matmul(F.normalize(a, p=p, dim= dim), F.normalize(b, p=p, dim= dim).T)

def cosine_similarity(a: torch.Tensor, b: torch.Tensor, p: int= 2, dim: int= 1):
    """
    a.shape = (*, dim)
    b.shape = (*, dim)
    a.dim == b.dim

    return diag of matrix
    """
    return cosine_matrix(a, b, p, dim).diag()

class RMSELoss(nn.Module):
    def __init__(self, eps=1e-6):
        super().__init__()
        self.mse = nn.MSELoss()
        self.eps = eps

    def forward(self, input: torch.Tensor, target: torch.Tensor):
        loss = torch.sqrt(self.mse(input, target) + self.eps)
        return loss

CRITERION = {'mse': nn.MSELoss,
             'mae': nn.L1Loss,
             'bce': nn.BCEWithLogitsLoss,
             'crs': nn.CrossEntropyLoss,
             'rmse': RMSELoss}


def euclid_distance(a: torch.Tensor, b: torch.Tensor, p: int= 2, dim: int= 1):
    """
    a.shape = (*, dim)
    b.shape = (*, dim)
    a.dim == b.dim

    return norm vector
    """
    return torch.norm(a - b, p= p, dim= dim)


if __name__ == '__main__':
      a = torch.randn(3, 128)
      b = torch.randn(4, 128)

      tmp = cosine_matrix(a, b)
      print(tmp)

tensor([[-0.0741,  0.1183, -0.2023, -0.0021],
        [ 0.0837, -0.0818,  0.0504,  0.0357],
        [ 0.1158, -0.1204, -0.0305,  0.0752]])


## scripts/utils/preprocess.py

In [ ]:
import sys
from collections import Counter
from copy import deepcopy
from itertools import chain
import warnings
warnings.simplefilter('ignore')

import numpy as np
import pandas as pd
from joblib import Parallel, delayed
from molvs import standardize_smiles
from rdkit import Chem, RDLogger
from rdkit.Chem import AllChem, DataStructs
from tqdm import tqdm

# from utils.decompose import MolFromFragments, MolToBRICSfragments, MapNumsToAdj

lg = RDLogger.logger()
lg.setLevel(RDLogger.CRITICAL)


def smiles2mol(s):
    m = Chem.MolFromSmiles(s)
    if m is None:
        print(f'[ERROR] {s} is not valid.', flush= True)
    return m

def frag2ecfp(frag, radius: int= 2, n_bits: int= 2048, useChiral: bool= True, ignore_dummy: bool= False):
    if ignore_dummy:
        frag = Chem.RemoveHs(AllChem.ReplaceSubstructs(frag, Chem.MolFromSmiles('*'), Chem.MolFromSmiles('[H]'), replaceAll= True)[0])
    return AllChem.GetMorganFingerprintAsBitVect(frag, radius, n_bits, useChirality= useChiral)

def FragmentsToIndices(fragments_list: list, fragment_idxs: dict, verbose: int= 0):
    if verbose:
        return [[fragment_idxs[f] for f in frags] for frags in tqdm(fragments_list)]
    else:
        return [[fragment_idxs[f] for f in frags] for frags in fragments_list]


def MolsToBRICSfragments(mols: list, debug: bool= True,
                         useChiral: bool= True, ignore_double: bool= False,
                         minFragSize: int= 1, maxFragNums: int= 50, maxDegree: int= 32,
                         uni_fragments: list= None, asFragments: bool= False,):
    fragments_list = []
    bondtypes_list = []
    bondMapNums_list = []
    all_fragments = []
    recon_flag = []

    # decompose mol
    for i, mol in enumerate(mols):
        s = Chem.MolToSmiles(mol)
        # hasChiral = bool(Chem.FindMolChiralCenters(mol)) if useChiral else False
        hasChiral = bool(len(Chem.FindPotentialStereo(mol, flagPossible= False))) if useChiral else False
        frags, bond_types, bondMapNums = MolToBRICSfragments(mol, minFragSize= minFragSize, maxDegree= maxDegree, useChiral= useChiral, useStereo= ignore_double)
        min_size = minFragSize
        while len(frags) > maxFragNums:
            min_size += 1
            frags, bond_types, bondMapNums = MolToBRICSfragments(mol, minFragSize= minFragSize, maxDegree= maxDegree, useChiral= hasChiral, useStereo= ignore_double)
            print(f"{s} has over {maxFragNums} fragments. min_size:{min_size-1} -> {min_size}", flush= True)
        if debug:
            try:
                adj = MapNumsToAdj(bondMapNums, bond_types)
                s1 = Chem.MolToSmiles(mol, isomericSmiles= hasChiral)
                s2 = Chem.MolToSmiles(MolFromFragments(frags, adj, asMol= True), isomericSmiles= hasChiral)
                if (s1 != s2) & (s1 != standardize_smiles(s2)):
                    if hasChiral:
                        s1_dash = Chem.CanonSmiles(s1, useChiral= 0)
                        s2_dash = Chem.CanonSmiles(s2, useChiral= 0)
                        if (s1_dash == s2_dash) | (s1_dash == standardize_smiles(s2_dash)):
                            recon = 2
                            print(f"[{i}] {s1}, {s2} is 3D unreconstructable.", flush= True)
                        else:
                            recon = 0
                            print(f"[{i}] {s1_dash}, {s2_dash} is 2D unreconstructable.", flush= True)
                    else:
                        recon = 0
                        print(f"[{i}] {s1}, {s2} is unreconstructable.", flush= True)
                else:
                    recon = 3 if hasChiral else 1
            except:
                recon = 0
                print(f'[{i}] {s} is an ERROR.')

        if recon > 0:
            all_fragments += frags
            fragments_list.append(frags)
            bondtypes_list.append(bond_types)
            bondMapNums_list.append(bondMapNums)
            recon_flag.append(recon)

    # calculate frequency and Uniqueness
    if uni_fragments is None:
        frag_freq = Counter(all_fragments)
        uni_fragments, freq_list = map(list, zip(*frag_freq.items()))
        df = pd.DataFrame({'SMILES': uni_fragments, 'frequency': freq_list})
        df = df.assign(length= df.SMILES.str.len())
        df = df.sort_values(['frequency', 'length'], ascending= [False, True])
        uni_fragments = df.SMILES.tolist()
        freq_list = df.frequency.tolist()
        uni_fragments = ['*'] + uni_fragments
        fragments_list = [0] + freq_list
    else:
        freq_list = None

    if not asFragments:
        fragment_idxs = dict(zip(uni_fragments, range(len(uni_fragments))))
        fragments_list = FragmentsToIndices(fragments_list, fragment_idxs)

    return fragments_list, bondtypes_list, bondMapNums_list, recon_flag, uni_fragments, freq_list


def debugMolToBRICSfragments(mol,
                             useChiral: bool= True, ignore_double: bool= False,
                             minFragSize: int= 1, maxFragNums: int= 50, maxDegree: int= 32):
    recon = 1
    iters = 0
    max_iters = 30
    try:
        # decompose
        s = Chem.MolToSmiles(mol) if mol is not None else None
        Chem.FindMolChiralCenters(mol)  # assign chirality
        hasChiral = bool(len(Chem.FindPotentialStereo(mol, flagPossible= False))) if useChiral else False
        frags, bond_types, bondMapNums = MolToBRICSfragments(mol, minFragSize= minFragSize, maxDegree= maxDegree, useChiral= hasChiral, useStereo= ignore_double)
        while (len(frags) > maxFragNums):
            iters += 1
            minFragSize += 1
            frags, bond_types, bondMapNums = MolToBRICSfragments(mol, minFragSize= minFragSize, maxDegree= maxDegree, useChiral= hasChiral, useStereo= ignore_double)
            # print(f"{s} has over {maxFragNums} fragments. min_size:{minFragSize-1} -> {minFragSize}", flush= True
            if iters > max_iters:
                raise ValueError(f'Over max iteration; {max_iters}. Remove it or increse max_nfrags.')

        # reconstruct
        adj = MapNumsToAdj(bondMapNums, bond_types)
        s1 = Chem.MolToSmiles(mol, isomericSmiles= hasChiral)
        s2 = Chem.MolToSmiles(MolFromFragments(frags, adj, asMol= True), isomericSmiles= hasChiral)
        if (s1 != s2) & (s1 != standardize_smiles(s2)):
            if hasChiral:
                s1_dash = Chem.CanonSmiles(s1, useChiral= 0)
                s2_dash = Chem.CanonSmiles(s2, useChiral= 0)
                if (s1_dash == s2_dash) | (s1_dash == standardize_smiles(s2_dash)):
                    recon = 2
                    # print(f"{s1}, {s2} is 3D unreconstructable.", flush= True)
                else:
                    recon = 0
                    print(f"{s1_dash}, {s2_dash} is 2D unreconstructable.", flush= True)
            else:
                recon = 0
                print(f"{s}, {s2} is unreconstructable.", flush= True)
        else:
            recon = 3 if hasChiral else 1
    except Exception as e:
        recon = 0
        print(f'{s} is an ERROR; {str(e)}', flush= True)

    if recon == 0:
        frags, bond_types, bondMapNums = None, None, None

    return frags, bond_types, bondMapNums, recon


def parallelMolsToBRICSfragments(mols: list,
                                 useChiral: bool= True, ignore_double: bool= False,
                                 minFragSize: int= 1, maxFragNums: int= 50, maxDegree: int= 32,
                                 df_frag: pd.DataFrame= {}, asFragments: bool= False,
                                 n_jobs: int= -1, verbose: int= 0):
    # decompose mol
    results = Parallel(n_jobs= n_jobs, verbose= verbose)(
        delayed(debugMolToBRICSfragments)(mol, minFragSize= minFragSize, maxFragNums= maxFragNums, maxDegree= maxDegree, useChiral= useChiral, ignore_double= ignore_double) for mol in mols)
    fragments_list, bondtypes_list, bondMapNums_list, recon_flag = zip(*results)
    fragments_list, bondtypes_list, bondMapNums_list, recon_flag = list(fragments_list), list(bondtypes_list), list(bondMapNums_list), list(recon_flag)

    # remove None
    for _ in range(fragments_list.count(None)):
        fragments_list.remove(None)
        bondtypes_list.remove(None)
        bondMapNums_list.remove(None)

    # calculate frequency and Uniqueness
    all_fragments = list(chain.from_iterable(fragments_list))
    frag_freq = Counter(all_fragments)
    uni_fragments, freq_list = map(list, zip(*frag_freq.items()))
    df = pd.DataFrame({'SMILES': uni_fragments, 'frequency': freq_list})
    df = df.assign(length= df.SMILES.str.len())
    df = df.sort_values(['frequency', 'length'], ascending= [False, True])

    # merge origin fragments list
    if len(df_frag):
        df = pd.concat([df_frag, df]).drop_duplicates(subset= 'SMILES', keep= 'first').reset_index(drop= True)
        uni_fragments = df.SMILES.tolist()
        freq_list = df.frequency.tolist()
    else:
        uni_fragments = df.SMILES.tolist()
        freq_list = df.frequency.tolist()
        uni_fragments = ['*'] + uni_fragments
        freq_list = [0] + freq_list

    # index
    if not asFragments:
        fragment_idxs = dict(zip(uni_fragments, range(len(uni_fragments))))
        fragments_list = FragmentsToIndices(fragments_list, fragment_idxs, verbose= verbose)

    return fragments_list, bondtypes_list, bondMapNums_list, recon_flag, uni_fragments, freq_list


def _smilesToMorganFingarPrintsAsBitVect(smiles, radius: int, n_bits: int, useChiral: bool= True, ignore_dummy: bool= True):
    ecfp = frag2ecfp(Chem.MolFromSmiles(smiles), radius, n_bits, useChiral= useChiral, ignore_dummy= ignore_dummy)
    ecfp_bits = np.array(ecfp, dtype= int).tolist()

    return ecfp, ecfp_bits


def SmilesToMorganFingetPrints(fragments: list, n_bits: int, dupl_bits: int= 0, radius: int= 2,
                               ignore_dummy: bool= True, useChiral: bool= True, n_jobs: int= 20):
    """
    fragments: a list of fragment smiles.
    distinct_ecfp: If fragments list includes same ecfp, add bits to distinguish ecfp.
    """
    results = Parallel(n_jobs= n_jobs)(delayed(_smilesToMorganFingarPrintsAsBitVect)(f, radius, n_bits, useChiral= useChiral, ignore_dummy= ignore_dummy) for f in fragments)
    ecfps, ecfp_list = map(list, zip(*results))

    if dupl_bits > 0:
        _, indices = np.unique(ecfp_list, axis= 0, return_index= True)
        uni_ecfps = [ecfps[i] for i in indices]
        dupl_idxs = []
        for e in uni_ecfps:
            dupl = np.where(np.array(DataStructs.BulkTanimotoSimilarity(e, ecfps)) == 1)[0]
            if len(dupl) >= 2:
                dupl_idxs.append(sorted(dupl.tolist()))

        if len(dupl_idxs) != 0:
            # add bits to ecfp
            max_len = max(list(map(len, dupl_idxs)))
            if max_len > 2**dupl_bits:
                raise ValueError(f'the number of duplicated ecfp {max_len} is greater than 2**{dupl_bits}')
            bits = [list(map(lambda x: int(x), list(f"{i:0{dupl_bits}b}"))) for i in range(max_len)]
            distinct_ecfp_list = [ecfp_list + bits[0] for ecfp_list in ecfp_list]
            for dupl in dupl_idxs:
                for i, d in enumerate(dupl):
                    distinct_ecfp_list[d] = distinct_ecfp_list[d][:-dupl_bits] + bits[i]
            ecfp_list = distinct_ecfp_list
        print(f'the number of duplicated ecfp is {len(dupl_idxs)}', flush= True)

    return ecfp_list


def IndicesToFeatures(indices_mols: list, features: list):
    mol_features_list = []
    for indices in indices_mols:
        mol_features_list.append([features[i] for i in indices])

    return mol_features_list


if __name__ == '__main__':
    smi = '[2H]C([2H])([2H])c1ccnc2c1NC(=O)c1cccnc1N2C1CC1'
    frag, bond, maps, recon = debugMolToBRICSfragments(Chem.MolFromSmiles(smi), ignore_double= False)
    adj = MapNumsToAdj(maps, bond)
    smi_rev = MolFromFragments(frag, adj)
    print(smi)
    print(smi_rev)

[2H]C([2H])([2H])c1ccnc2c1NC(=O)c1cccnc1N2C1CC1
[2H]C([2H])([2H])c1ccnc2c1NC(=O)c1cccnc1N2C1CC1


## scripts/utils/standardize_smiles.py

In [ ]:
import argparse
import copy
import random
import pandas as pd
import numpy as np
from joblib import Parallel, delayed

from rdkit import Chem
from rdkit.Chem import Descriptors, AllChem

# import moses
from molvs import  Standardizer


# parser = argparse.ArgumentParser(description= 'please enter paths')
parser = argparse.ArgumentParser()
parser.add_argument('--data_path', type= str, help= 'csv file', default="/content/FRATTVAE/data/example.csv")
parser.add_argument('--keep_dupls', action= 'store_true', default=True)
parser.add_argument('--n_jobs', type= int, default= 24, help= 'the number of cpu for parallel, default 24')
args, unknown = parser.parse_known_args()
data_path = args.data_path
df = pd.read_csv(data_path)
if 'SMILES' not in df.columns:
    raise ValueError('Please change the name of column smiles to "SMILES"')
fname = data_path.rsplit('.csv')[0]
print(f'loaded: {args.data_path}', flush= True)

stand = Standardizer()

def clearAtomMapNums(mol):
    for atom in mol.GetAtoms():
        atom.SetAtomMapNum(0)

def standardize_and_metrics(smi):
    try:
        mol = Chem.MolFromSmiles(smi)
        mol = copy.deepcopy(mol)
        clearAtomMapNums(mol)
        Chem.SanitizeMol(mol)
        mol = Chem.RemoveHs(mol)
        # mol = stand.disconnect_metals(mol)
        mol = stand.normalize(mol)
        mol = stand.reionize(mol)
        # Chem.AssignStereochemistry(mol, force=True, cleanIt=True)
        smi_std = Chem.MolToSmiles(mol)

        natom = mol.GetNumAtoms()
        # mw = moses.metrics.weight(mol)
        # logP = moses.metrics.logP(mol)
        # qed = moses.metrics.QED(mol)
        # sa = moses.metrics.SA(mol)
        # npl = moses.metrics.NP(mol)
        tpsa = Chem.Descriptors.TPSA(mol)
        ct = Descriptors.BertzCT(mol)
    except:
        mol = smi_std = None
        # natom = mw = logP = qed = sa = npl = tpsa = ct =  None
        natom = tpsa = ct =  None

    if smi_std is None:
        print('none', flush= True)

    # return smi, smi_std, natom, mw, logP, qed, sa, npl, tpsa, ct
    return smi, smi_std, natom, tpsa, ct

# standardize
results = Parallel(n_jobs= args.n_jobs)(delayed(standardize_and_metrics)(s) for s in df.SMILES)
# smiles, smiles_std, natoms_list, mw_list, logP_list, qed_list, sa_list, npl_list, tpsa_list, ct_list = map(list, zip(*results))
smiles, smiles_std, natoms_list, tpsa_list, ct_list = map(list, zip(*results))

# train-test split
assert len(df) == len(smiles_std)
try:
    test = df.test.tolist()
except:
    random.seed(0)
    test = random.choices([0, 1, -1], k= len(df), weights= [0.90, 0.05, 0.05])
    print('random train-valid-test split. train:valid:test= 0.90:0.05:0.05', flush= True)

# add columns
df['SMILES'] = smiles_std
df['test'] = test
# df_tmp = pd.DataFrame({'natoms': natoms_list, 'MW': mw_list, 'logP': logP_list, 'QED': qed_list,
#                        'SA': sa_list, 'NP': npl_list, 'TPSA': tpsa_list, 'BertzCT': ct_list})

df_tmp = pd.DataFrame({'natoms': natoms_list, 'TPSA': tpsa_list, 'BertzCT': ct_list})
df_new = pd.concat([df, df_tmp], axis= 1)

# remove errors
df_error = df_new.loc[df_new['SMILES'].isnull()]
n_errors = len(df_error)
df_new = df_new.loc[~df_new['SMILES'].isnull()].reset_index(drop= True)

# check duplications
dupls = df_new.duplicated(subset= 'SMILES', keep= 'first')
n_dupls = sum(dupls)
if not args.keep_dupls:
    df_error = pd.concat([df_error, df_new.loc[dupls]])
    df_new = df_new.loc[~dupls].reset_index(drop= True)

# save
df_new.to_csv(f'{fname}_standardized.csv', index= False)
if len(df_error) > 0:
    df_error.to_csv(f'{fname}_removed.csv', index= False)

# output
with open(f'{fname}_stats.txt', 'a') as f:
    for col in df_tmp.columns:
        f.write(f'-{col}: {np.nanmean(df_tmp[col]):.4f}±{np.nanstd(df_tmp[col]):.4f} ({np.nanmin(df_tmp[col]):.4f}-{np.nanmax(df_tmp[col]):.4f})')

print(f'[BEFORE] {len(df)}', flush= True)
print(f'[AFTER] {len(df_new)}', flush= True)
print(f'removed:  {len(df_error)} (error: {n_errors}, dupl: {n_dupls}, keep_dupls: {args.keep_dupls})', flush= True)

loaded: /content/FRATTVAE/data/example.csv
[BEFORE] 500
[AFTER] 500
removed:  0 (error: 0, dupl: 0, keep_dupls: True)


In [ ]:
# Set the DGLBACKEND environment variable to 'pytorch'
os.environ['DGLBACKEND'] = 'pytorch'

# (Optional) Verify the setting
print(f"DGL backend set to: {os.environ.get('DGLBACKEND')}")

DGL backend set to: pytorch


## scripts/utils/tree.py

In [ ]:
import torch
import dgl
import numpy as np

import warnings
warnings.filterwarnings("ignore", message="Recommend creating")

"""
Reference: https://github.com/microsoft/icecaps/blob/master/icecaps/util/trees.py
           https://github.com/inyukwo1/tree-lstm/blob/master/tree_lstm/tree_lstm.py (2023/07/07)
"""

class FragmentTree:
    def __init__(self, dgl_graph= None):
        self.dgl_graph = dgl_graph if dgl_graph else dgl.DGLGraph()
        self.max_depth = 0
        self.max_degree = 0

    def add_node(self, parent_id=None, feature: torch.Tensor = torch.Tensor(), fid: int= -1, bondtype: int= 1, data: dict= None):
        if data is None:
            data = {'x': feature.unsqueeze(0),
                    'fid': torch.tensor([fid]).unsqueeze(0)}
        self.dgl_graph.add_nodes(1, data= data)
        added_node_id = self.dgl_graph.number_of_nodes() - 1

        if parent_id is not None:
            self.dgl_graph.ndata['level'][added_node_id] = self.dgl_graph.ndata['level'][parent_id] + 1
            self.dgl_graph.add_edges(added_node_id, parent_id, data= {'w': torch.tensor([bondtype]).unsqueeze(0)})
            self.max_degree = max(self.max_degree, len(self.dgl_graph.predecessors(parent_id)))
        elif added_node_id > 0:
            self.dgl_graph.ndata['level'][added_node_id] = torch.tensor([0]).int()
        else:
            self.dgl_graph.ndata['level'] = torch.tensor([0]).int()

        self.max_depth = self.dgl_graph.ndata['level'].max().item()
        # self.max_width = max([self.width(level) for level in range(self.max_depth+1)]).item()

        return added_node_id

    def add_link(self, child_id, parent_id, bondtype: int= 1):
        self.dgl_graph.add_edges(child_id, parent_id, data= {'w': torch.tensor([bondtype]).unsqueeze(0)})

    def remove_node(self, node_id: int):
        self.dgl_graph.remove_nodes(node_id)

    def remove_edge(self, edge_id: int):
        self.dgl_graph.remove_edges(edge_id)

    def adjacency_matrix(self):
        n_node = self.dgl_graph.num_nodes()
        if n_node < 2:
            adj = torch.tensor([[0]])
        else:
            indices = torch.stack(self.dgl_graph.all_edges())
            values = self.dgl_graph.edata['w'].squeeze()
            adj = torch.sparse_coo_tensor(indices, values, size= (n_node, n_node)).to_dense()

        return adj

    def to(self, device: str= 'cpu'):
        self.dgl_graph = self.dgl_graph.to(device)
        return self

    def reverse(self):
        self.dgl_graph = dgl.reverse(self.dgl_graph, copy_ndata= True, copy_edata= True)
        return self

    def set_all_positional_encoding(self, d_pos: int= None, n: int= None):
        """
        if n is not None, encoding as all nodes have n children.
        """
        d_pos = d_pos if d_pos else self.max_depth * self.max_degree
        self.dgl_graph.ndata['pos'] = torch.zeros(self.dgl_graph.num_nodes(), d_pos)
        for nid in self.dgl_graph.nodes()[1:]:
            parent = self.dgl_graph.successors(nid)
            if len(parent) > 0:
                parent = parent[0]
            else:
                continue
            children = self.dgl_graph.predecessors(parent).tolist()

            n = n if n else len(children)
            assert n >= len(children)
            positional_encoding = [0.0 for _ in range(n)]
            positional_encoding[children.index(nid)] = 1.0
            positional_encoding += self.dgl_graph.ndata['pos'][parent].tolist()

            self.dgl_graph.ndata['pos'][nid] = torch.tensor(positional_encoding)[:d_pos]

    def set_positional_encoding(self, nid: int, num_sibling: int= None, d_pos: int= None):
        d_pos = d_pos if d_pos else self.max_depth * self.max_degree

        parents = self.dgl_graph.successors(nid)
        if len(parents) == 0:
            self.dgl_graph.ndata['pos'] = torch.zeros(self.dgl_graph.num_nodes(), d_pos)
            positional_encoding = [0.0] * d_pos
        else:
            parent = parents[0]
            sibling = self.dgl_graph.predecessors(parent).tolist()
            num_sibling = num_sibling if num_sibling is not None else len(sibling)
            assert num_sibling >= len(sibling)

            positional_encoding = [0.0 for _ in range(num_sibling)]
            positional_encoding[sibling.index(nid)] = 1.0
            positional_encoding += self.dgl_graph.ndata['pos'][parent].tolist()

        self.dgl_graph.ndata['pos'][nid] = torch.tensor(positional_encoding)[:d_pos]

    def width(self, level: int):
        return (self.dgl_graph.ndata['level'] == level).sum()


class BatchedFragmentTree:
    def __init__(self, tree_list, max_depth: int= None, max_degree: int= None):
        graph_list = []
        depth_list, degree_list = zip(*[(tree.max_depth, tree.max_degree) for tree in tree_list])
        if (max_depth is None) | (max_degree is None):
            self.max_depth = max(depth_list)
            self.max_degree = max(degree_list)
        else:
            if (max_depth < max(depth_list)) | (max_degree < max(degree_list)):
                print(f'[WARNING] max depth:{max_depth} < {max(depth_list)} or max degree:{max_degree} < {max(degree_list)}', flush= True)
            self.max_depth = max_depth
            self.max_degree = max_degree

        for tree in tree_list:
            tree.set_all_positional_encoding(d_pos= self.max_depth * self.max_degree)
            graph_list.append(tree.dgl_graph)
        self.batch_dgl_graph = dgl.batch(graph_list)

    def get_ndata(self, key: str, node_ids: list= None, pad_value: int= 0):
        graph_list = dgl.unbatch(self.batch_dgl_graph)
        ndatas = []
        max_nodes_num = max([graph.num_nodes() for graph in graph_list])
        for i, graph in enumerate(graph_list):
            if node_ids:
                node_id = node_ids[i] if i < len(node_ids) else node_ids[0]
                states = graph.ndata[key][node_id]
            else:
                states = graph.ndata[key]
                node_num, state_num = states.size()
                if len(states) < max_nodes_num:
                    padding = states.new_full((max_nodes_num - node_num, state_num), pad_value)
                    states = torch.cat((states, padding), dim=0)
            ndatas.append(states)
        return torch.stack(ndatas)

    def get_edata(self, key: str= 'w', edge_ids: list= None, pad_value: int= 0):
        graph_list = dgl.unbatch(self.batch_dgl_graph)
        edatas = []
        max_edges_num = max([graph.num_edges() for graph in graph_list])
        for i, graph in enumerate(graph_list):
            if edge_ids:
                edge_id = edge_ids[i] if i < len(edge_ids) else edge_ids[0]
                states = graph.edata[key][edge_id]
            else:
                states = graph.edata[key]
                edge_num, state_num = states.size()
                if len(states) < max_edges_num:
                    padding = states.new_full((max_edges_num - edge_num, state_num), pad_value)
                    states = torch.cat((states, padding), dim=0)
            edatas.append(states)
        return torch.stack(edatas)

    def get_tree_list(self):
        graph_list = dgl.unbatch(self.batch_dgl_graph)
        return [FragmentTree(dgl_graph= graph) for graph in graph_list]

    def to(self, device: str= 'cpu'):
        self.batch_dgl_graph = self.batch_dgl_graph.to(device)
        return self

    def reverse(self):
        reverse_graph_list = [dgl.reverse(g, copy_ndata= True, copy_edata= True) for g in dgl.unbatch(self.batch_dgl_graph)]
        self.batch_dgl_graph = dgl.batch(reverse_graph_list)

        return self


def make_tree(frag_indices: list, ecfps: torch.Tensor, bond_types: list, bondMapNums: list, d_pos: int= None) -> FragmentTree:
    """
    frag_indices: a list of fragments indices
    ecfps: ecfps of fragments, shape= (len(frag_indices), n_bits)
    bond_types: a list of bondtype (1: single, 2: double, 3: triple)
    bondMapNums: a list of connection order lists.
                 ex. [[1], [1, 2], [2]] -> first connect frag0 and 1, next frag1 and 2.
    d_pos: dimension of positional encoding
    """
    if type(ecfps) == list:
        ecfps = torch.tensor(ecfps).float()

    tree = FragmentTree()
    tree.add_node(parent_id= None, feature= ecfps[0], fid= frag_indices[0], bondtype= 0)

    stack = [0]
    node_ids = [0] * len(frag_indices)
    while max(map(len, bondMapNums)) > 0:
        if stack:
            parent = stack[-1]
            pid = node_ids[parent]
            if bondMapNums[parent]:
                b = bondMapNums[parent].pop(0)
            else:
                stack.pop(-1)
                continue
        else:
            # accept partial trees
            idx = [i for i in range(len(frag_indices)) if len(bondMapNums[i]) > 0][0]
            stack.append(idx)
            add_node_id = tree.add_node(parent_id= None, feature= ecfps[idx], fid= frag_indices[idx], bondtype= 0)
            node_ids[idx] = add_node_id
            continue

        child_list = [b in mapnums for mapnums in bondMapNums]
        if np.any(child_list):
            c = child_list.index(True)
            add_node_id = tree.add_node(parent_id= pid, feature= ecfps[c], fid= frag_indices[c], bondtype= bond_types[b-1])
            node_ids[c] = add_node_id
            stack.append(c)

    if d_pos:
        tree.set_all_positional_encoding(d_pos)

    return tree


def get_tree_features(frag_indices: list, ecfps: torch.Tensor, bond_types: list, bondMapNums: list,
                      max_depth: int= None, max_degree: int= None, free_n: bool= False):
    tree = make_tree(frag_indices, ecfps, bond_types, bondMapNums)

    max_depth = max_depth if max_depth else tree.max_depth
    max_degree = max_degree if max_degree else tree.max_degree

    if (max_depth < tree.max_depth) | (max_degree < tree.max_degree):
        print(f'[WARNING] max depth:{max_depth} < {tree.max_depth} or max degree:{max_degree} < {tree.max_degree}', flush= True)

    n = None if free_n else max_degree
    tree.set_all_positional_encoding(d_pos= max_depth * max_degree, n= n)
    fids = tree.dgl_graph.ndata['fid'].squeeze(-1)
    positions = tree.dgl_graph.ndata['pos']
    features = tree.dgl_graph.ndata['x']

    return fids, features, positions


def get_pad_features(tree_list, key: str, max_nodes_num: int):
    ndatas = []
    for tree in tree_list:
        states = tree.dgl_graph.ndata[key]
        node_num, state_num = states.size()
        if len(states) < max_nodes_num:
            padding = states.new_full((max_nodes_num - node_num, state_num), 0)
            states = torch.cat((states, padding), dim=0)
        ndatas.append(states)
    return torch.stack(ndatas)


if __name__ == '__main__':
    frag_indice = [0, 1, 2, 3]
    ecfps = torch.ones(4, 12).float()
    bondtypes = [1, 1, 1]
    bondMapNum = [[1], [1, 2], [2, 3]]

    tree = make_tree(frag_indice, ecfps, bondtypes, bondMapNum, d_pos= 16)
    print(tree)

## scripts/models/frattvae.py

In [ ]:
import numpy as np
from joblib import Parallel, delayed

import torch
import torch.nn as nn

# from utils.tree import FragmentTree, get_pad_features
# from utils.mask import generate_square_subsequent_mask
# from utils.construct import constructMol, constructMolwithTimeout


class TreePositionalEncoding(nn.Module):
    """
    Reference: https://github.com/microsoft/icecaps/blob/master/icecaps/estimators/abstract_transformer_estimator.py
    """
    def __init__(self, d_model: int, d_pos: int, depth: int, width: int) -> None:
        super().__init__()
        self.d_params = d_pos // (depth * width)
        self.d_model = d_model
        self.depth = depth
        self.width = width
        self.params = nn.Parameter(torch.randn(self.d_params), requires_grad= True)
        self.fc = nn.Linear(self.d_params * self.depth * self.width, d_model)

        # initialization
        self.tree_weights = None

    def forward(self, positions: torch.Tensor):
        """
        positions: shape= (Batch_size, Length, depth * width)
        """
        if self.training | (self.tree_weights is None):
            self._update_weights()
        treeified = positions.unsqueeze(-1) * self.tree_weights.to(positions.device)     # (B, L, depth*width, d_param)
        treeified = treeified.flatten(start_dim= 2)                                      # (B, L, depth*width*d_param = d_pos)
        if treeified.shape[-1] != self.d_model:
            treeified = self.fc(treeified)

        return treeified

    def _update_weights(self):
        params = torch.tanh(self.params)
        tiled_tree_params = params.view(1, 1, -1).repeat(self.depth, self.width, 1)
        tiled_depths = torch.arange(self.depth, dtype=torch.float32, device= params.device).view(-1, 1, 1).repeat(1, self.width, self.d_params)
        tree_norm = torch.sqrt((1 - params.square()) * self.d_model / 2)
        self.tree_weights = (tiled_tree_params ** tiled_depths) * tree_norm
        self.tree_weights = self.tree_weights.view(self.depth * self.width, self.d_params)


class FRATTVAE(nn.Module):
    def __init__(self, num_tokens: int, depth: int, width: int,
                 feat_dim: int= 2048, latent_dim: int= 256,
                 d_model: int= 512, d_ff: int= 2048, num_layers: int= 6, nhead: int= 8,
                 activation: str= 'gelu', dropout: float= 0.1, n_jobs: int= 4) -> None:
        super().__init__()
        assert activation in ['relu', 'gelu']
        self.d_model = d_model
        self.latent_dim = latent_dim
        self.dropout = dropout
        self.depth = depth
        self.width = width
        self.n_jobs = n_jobs

        self.embed = nn.Embedding(num_embeddings= 1, embedding_dim= d_model)      # <root>
        self.fc_ecfp = nn.Sequential(nn.Linear(feat_dim, feat_dim//2),
                                     nn.Linear(feat_dim//2, d_model))
        self.PE = TreePositionalEncoding(d_model= d_model, d_pos= max(d_model, depth*width), depth= depth, width= width)

        # transformer encoder
        encoder_layer = nn.TransformerEncoderLayer(d_model= d_model, nhead= nhead, dim_feedforward= d_ff,
                                                   dropout= self.dropout, activation= activation, batch_first= True)
        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers= num_layers)

        # vae
        self.fc_vae = nn.Sequential(nn.Linear(d_model, latent_dim),
                                    nn.Linear(latent_dim, 2*latent_dim))

        # transformer decoder
        self.fc_memory = nn.Linear(latent_dim, d_model)
        decoder_layer = nn.TransformerDecoderLayer(d_model= d_model, nhead= nhead, dim_feedforward= d_ff,
                                                   dropout= self.dropout, activation= activation, batch_first= True)
        self.decoder = nn.TransformerDecoder(decoder_layer, num_layers= num_layers)
        self.fc_dec = nn.Linear(d_model, num_tokens)

        # for decode smiles
        self.labels = None

    def forward(self, features: torch.Tensor, positions: torch.Tensor,
                src_mask: torch.Tensor= None, src_pad_mask: torch.Tensor= None,
                tgt_mask: torch.Tensor= None, tgt_pad_mask: torch.Tensor= None,
                frag_ecfps: torch.Tensor= None, ndummys: torch.Tensor= None,
                max_nfrags: int= 20, free_n: bool= False, sequential: bool= None, conditions: torch.Tensor= None):
        """
        encode and decode
        Decode in parallel when training process.
        """
        sequential = not self.training if sequential is None else sequential

        z, mu, ln_var = self.encode(features, positions, src_mask, src_pad_mask, conditions)
        if sequential:
            output = self.sequential_decode(z, frag_ecfps, ndummys, conditions= conditions, max_nfrags= max_nfrags, free_n= free_n)
        else:
            output = self.decode(z, features, positions, tgt_mask, tgt_pad_mask, conditions)

        return z, mu, ln_var, output


    def encode(self, features: torch.Tensor, positions: torch.Tensor,
               src_mask: torch.Tensor= None, src_pad_mask: torch.Tensor= None, conditions: torch.Tensor= None):
        """
        features: shape= (Batch_size, Length, feat_dim)
        positions: shape= (Batch_size, Length, depth * width)
        condtions: shape= (Batch_size, num_conditions, d_model) if conditional vae.
        src_mask: source mask for masked attention, shape= (Length+num_conditions+1, Length+num_conditions+1)
        src_pad_mask: shape = (Batch_size, Length+num_conditions+1)
        """
        num_conditions = conditions.shape[1] if conditions is not None else 0

        # positional embbeding
        src = self.fc_ecfp(features) + self.PE(positions)           # (B, L, d_model),  * math.sqrt(self.d_model)?

        # attach super root
        root_embed = self.embed(src.new_zeros(src.shape[0], 1).long())
        if num_conditions > 0:
            src = torch.cat([conditions, root_embed, src], dim= 1)  # (B, L+num_conditions+1, d_model)
        else:
            src = torch.cat([root_embed, src], dim= 1)              # (B, L+1, d_model)

        # transformer encoding
        out = self.encoder(src, mask= src_mask, src_key_padding_mask= src_pad_mask)
        out = out[:, num_conditions, :].squeeze(1)

        # vae
        mu, ln_var = self.fc_vae(out).chunk(2, dim= -1)
        z = self.reparameterization_trick(mu, ln_var)               # (B, latent_dim)

        return z, mu, ln_var


    def decode(self, z: torch.Tensor, features: torch.Tensor, positions: torch.Tensor,
               tgt_mask: torch.Tensor= None, tgt_pad_mask: torch.Tensor= None, conditions: torch.Tensor= None):
        """
        z: encoder output. shape= (Batch_size, latent_dim)
        features: shape= (Batch_size, Length, feat_dim)
        positions: shape= (Batch_size, Length, depth * width)
        condtions: shape= (Batch_size, num_conditions, d_model) if conditional vae.
        tgt_mask: target mask for masked attention, shape= (Length+1, Length+1)
        tgt_pad_mask: target mask for padding, shape= (Batch_size, Length+1)

        output: logits of label preditions, shape= (Batch_size, Length+1, num_labels)
        """
        num_conditions = conditions.shape[1] if conditions is not None else 0

        # latent variable to memory
        memory = self.fc_memory(z).unsqueeze(1)                     # (B, 1, d_model)

        # postional embedding
        tgt = self.fc_ecfp(features) + self.PE(positions)           # (B, L, d_model)

        # attach supur root
        root_embed = self.embed(tgt.new_zeros(tgt.shape[0], 1).long())
        if num_conditions > 0:
            tgt = torch.cat([conditions, root_embed, tgt], dim= 1)  # (B, L+num_conditions+1, d_model)
        else:
            tgt = torch.cat([root_embed, tgt], dim= 1)              # (B, L+1, d_model)

        # transformer decoding
        out = self.decoder(tgt, memory, tgt_mask= tgt_mask, tgt_key_padding_mask= tgt_pad_mask)
        out = self.fc_dec(out[:, num_conditions:])                  # (B, L+1, num_tokens)

        return out


    def sequential_decode(self, z: torch.Tensor, frag_ecfps: torch.Tensor, ndummys: torch.Tensor,
                          max_nfrags: int= 30, free_n: bool= False, asSmiles: bool= False, conditions: torch.Tensor= None) -> list:
        """
        z: latent variable. shape= (Batch_size, latent_dim)
        frag_ecfps: fragment ecfps. shape= (num_labels, feat_dim)
        ndummys: The degree of a fragment means how many children it has. shape= (num_labels, )
        max_nfrags: the maximum number of fragments
        free_n: if False, tree positional encoding as all nodes have n children.

        output: list of fragment tree
        """
        batch_size = z.shape[0]
        device = z.device
        num_conditions = conditions.shape[1] if conditions is not None else 0

        # latent variabel to memory
        memory = self.fc_memory(z).unsqueeze(1)

        # root prediction
        root_embed = self.embed(torch.zeros(batch_size, 1, device= device).long())
        if num_conditions > 0:
            root_embed = torch.cat([conditions, root_embed], dim= 1)    # (B, num_conditions+1, d_model)
        tgt_pad_mask = torch.all(root_embed==0, dim= -1).to(device)     # (B, num_conditions+1)
        out = self.decoder(root_embed, memory, tgt_key_padding_mask= tgt_pad_mask)
        out = self.fc_dec(out[:, num_conditions:])
        root_idxs = out.argmax(dim= -1).flatten()      # (B, )
        # out = out.argsort(dim= -1, descending= True).squeeze(1)
        # root_idxs = torch.where(out[:,0]!=0, out[:,0], out[:,1])    # (B, )

        continues = []
        target_ids = [0] * batch_size
        target_ids_list = [[0] for _ in range(batch_size)]
        tree_list = [FragmentTree() for _ in range(batch_size)]
        for i, idx in enumerate(root_idxs):
            parent_id = tree_list[i].add_node(parent_id= None, feature= frag_ecfps[idx], fid= idx.item(), bondtype= 0)
            assert parent_id == 0
            tree_list[i].set_positional_encoding(parent_id, d_pos= self.depth * self.width)
            continues.append(ndummys[idx].item() > 0)

        nfrags = 1
        while (nfrags < max_nfrags) & (sum(continues) > 0):
            # features
            tgt_mask = generate_square_subsequent_mask(length= nfrags+num_conditions+1).to(device)
            tgt_mask[:, :num_conditions+1] = 0      # no sequence mask of conditions
            tgt_pad_mask = torch.hstack([tgt_pad_mask, tgt_pad_mask.new_full(size= (batch_size, 1), fill_value= False)])
            features = get_pad_features(tree_list, key= 'x', max_nodes_num= nfrags).to(device)
            positions = get_pad_features(tree_list, key= 'pos', max_nodes_num= nfrags).to(device)
            assert features.shape[0] == positions.shape[0]

            # forward
            tgt = self.fc_ecfp(features) + self.PE(positions)
            tgt = torch.cat([root_embed, tgt], dim= 1)          # (B, nfrags+num_conditions+1, d_model)

            out = self.decoder(tgt, memory, tgt_mask= tgt_mask, tgt_key_padding_mask= tgt_pad_mask)
            out = self.fc_dec(out[:, num_conditions:])              # (B, nfrags+1, num_labels)

            new_idxs = out[:, -1, :].argmax(dim= -1).flatten()      # (B,)

            # add node
            for i, idx in enumerate(new_idxs):
                if continues[i]:
                    if ndummys[idx] == 0:   # don't generate salt compounds.
                        idx = torch.tensor(0)
                    if idx != 0:
                        parent_id = target_ids[i]
                        add_node_id = tree_list[i].add_node(parent_id= parent_id, feature= frag_ecfps[idx], fid= idx.item(), bondtype= 1)
                        parent_fid = tree_list[i].dgl_graph.ndata['fid'][parent_id].item()
                        num_sibling = ndummys[parent_fid].item() - 1 if parent_id > 0 else ndummys[parent_fid].item()
                        if free_n:
                            tree_list[i].set_positional_encoding(add_node_id, num_sibling= num_sibling, d_pos= self.depth * self.width)
                        else:
                            tree_list[i].set_positional_encoding(add_node_id, num_sibling= self.width, d_pos= self.depth * self.width)
                        level = tree_list[i].dgl_graph.ndata['level'][add_node_id].item()

                        # compare the current number of siblings with the ideal number of siblings
                        if (len(tree_list[i].dgl_graph.predecessors(parent_id)) >= num_sibling):
                            target_ids_list[i].pop(-1)

                        # whether the node has children
                        if (ndummys[idx] > 1) & (self.depth > level):
                            target_ids_list[i].append(add_node_id)

                    continues[i] = bool(target_ids_list[i]) if (idx != 0) else False
                    target_ids[i] = target_ids_list[i][-1] if continues[i] else 0
            nfrags += 1

        if asSmiles:
            if self.labels is not None:
                # outputs = [constructMol(self.labels[tree.dgl_graph.ndata['fid'].squeeze(-1).tolist()], tree.adjacency_matrix().tolist()) for tree in tree_list]
                outputs = Parallel(n_jobs= self.n_jobs)(delayed(constructMol)(self.labels[tree.dgl_graph.ndata['fid'].squeeze(-1).tolist()], tree.adjacency_matrix().tolist()) for tree in tree_list)
                # outputs = Parallel(n_jobs= self.n_jobs, backend='threading')(delayed(constructMolwithTimeout)(self.labels[tree.dgl_graph.ndata['fid'].squeeze(-1).tolist()], tree.adjacency_matrix().tolist()) for tree in tree_list)
            else:
                raise ValueError('If asSmiles= True, please set labels. exaple; self.set_labels(labels)')
        else:
            outputs = tree_list

        return outputs

    def reparameterization_trick(self, mu, ln_var):
        eps = torch.randn_like(mu)
        z = mu + torch.exp(ln_var / 2) * eps if self.training else mu

        return z

    def set_labels(self, labels):
        if type(labels) == np.ndarray:
            self.labels = labels
        else:
            self.labels = np.array(labels)

## scripts/models/property.py

In [ ]:
import torch
import torch.nn as nn

class propLinear(nn.Module):
    def __init__(self, input_dim: int, output_dim: int, hidden_dim: int= 64) -> None:
        super().__init__()
        self.linear1 = nn.Linear(input_dim, hidden_dim)
        self.linear2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, z: torch.tensor):
        return self.linear2(self.linear1(z))


class propRank(nn.Module):
    def __init__(self, input_dim: int, output_dim: int) -> None:
        super().__init__()
        self.linear1 = nn.Linear(input_dim, 64)
        self.linear2 = nn.Linear(64, output_dim)
        self.sm = nn.Softmax(dim= 1)

    def forward(self, z: torch.tensor):
        x = self.linear2(self.linear1(z))
        return self.sm(x)


class PairWiseLoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.sp = nn.Softplus()

    def forward(self, s1: float, s2: float, t: float):
        # t = 1 if x1 > x2
        #     0 if x2 > x1
        #     0.5 otherwise
        o = s1 - s2
        return torch.mean(-t * o + self.sp(o))


PROPMDL = {
    'linear': propLinear,
    'rank': propRank
}

## unzip prerequisity available at https://drive.google.com/drive/folders/16LAR-wDdsNEAYbVT8KcG_DJtm6a7GhVP (FRATTVAE's authors)

In [ ]:
# # Google Colab に zip ファイルをアップロード
# from google.colab import files
# uploaded = files.upload()  # ファイル選択ダイアログが表示される

In [ ]:
import zipfile

# ZIPファイル名
# zip_file = "your_file.zip"
# zip_file = "/content/ZINC_JTVAE_standardized_struct.zip"
zip_file = "/content/Polymer_standardized_struct.zip"
# 解凍先フォルダ
extract_folder = "/content/extracted"

with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall(extract_folder)

print(f"unzipped at : {extract_folder}")

unzipped at : /content/extracted


In [ ]:
import os
os.getcwd()

'/content'

## scripts/process.py

In [ ]:
import os
import datetime
import gc
import pickle
import random
import time
import warnings
warnings.simplefilter('ignore')

import numpy as np
import pandas as pd
from joblib import Parallel, delayed

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence

from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem

# from utils.apps import second2date, torch_fix_seed
# from utils.mask import create_mask
# from utils.tree import get_pad_features, get_tree_features
# from utils.metrics import cosine_matrix, euclid_distance, CRITERION
# from utils.preprocess import debugMolToBRICSfragments
# from utils.construct import reconstructMol, constructMol, calc_tanimoto


def reconstruct(dataloader,
                smiles: list,
                labels: list,
                frag_ecfps: torch.Tensor,
                ndummys: torch.Tensor,
                model: nn.Module,
                pmodel: nn.Module= None,
                criterion= None,
                max_nfrags: int= 30,
                useChiral: bool= True,
                free_n: bool= False,
                n_jobs: int= -1,
                device: torch.device= torch.device('cpu'),
                seed: int= 0
                ):
    torch_fix_seed(seed)

    labels = np.array(labels)
    z_list, prop_list, pred_list = [], [], []
    frag_idxs_list, adjs_list = [], []
    label_acc, sims_list = 0, []
    s = time.time()
    with torch.no_grad():
        for i, data in enumerate(dataloader):
            frag_indices = data[0]
            features = frag_ecfps[frag_indices.flatten()].reshape(frag_indices.shape[0], frag_indices.shape[1], -1).to(device)
            positions = data[1].to(device)
            prop = data[2].to(device)

            # make mask
            src = torch.hstack([torch.full((frag_indices.shape[0], 1), -1), frag_indices.detach()]).to(device)  # for super root
            src_mask, _, src_pad_mask, _ = create_mask(src, src, pad_idx= 0, batch_first= True)

            # encode & decode
            z, _, _, tree_list = model(features, positions,
                                       src_mask= src_mask, src_pad_mask= src_pad_mask,
                                       frag_ecfps= frag_ecfps, ndummys= ndummys, max_nfrags= max_nfrags, free_n= free_n)

            # property prediction
            if pmodel:
                pred_prop = pmodel(z.to(device))
                prop_list.append(prop)
                pred_list.append(pred_prop)

            # stock in list
            z_list.append(z.cpu())
            frag_idxs, adjs = zip(*[(tree.dgl_graph.ndata['fid'].squeeze(-1).tolist(), tree.adjacency_matrix().tolist()) for tree in tree_list])
            frag_idxs_list += list(frag_idxs)
            adjs_list += list(adjs)

            # calc accuracy
            acc = 0
            for idxs, true_idxs in zip(frag_idxs, frag_indices):
                idxs = torch.tensor(idxs)
                idxs_pad, true_idxs_pad = pad_sequence([idxs, true_idxs], batch_first= True, padding_value= 0).chunk(2, dim= 0)
                acc += torch.mean(true_idxs_pad.eq(idxs_pad).float()).item()
            label_acc += acc / frag_indices.shape[0]

            # calc similarity of latent variables
            recon_indices = get_pad_features(tree_list, key= 'fid', max_nodes_num= max_nfrags).squeeze(-1)
            features = get_pad_features(tree_list, key= 'x', max_nodes_num= max_nfrags).to(device)
            positions = get_pad_features(tree_list, key= 'pos', max_nodes_num= max_nfrags).to(device)
            src = torch.hstack([torch.full((recon_indices.shape[0], 1), -1), recon_indices.detach()]).to(device)  # for super root
            src_mask, _, src_pad_mask, _ = create_mask(src, src, pad_idx= 0, batch_first= True)
            z_dash, _, _ = model.encode(features, positions, src_mask, src_pad_mask)
            cosines = cosine_matrix(z, z_dash).diag()
            sims_list += cosines.tolist()

            if ((i+1) % 10 == 0) | (i == 0) | ((i+1) == len(dataloader)):
                print(f'[{i+1:0=3}/{len(dataloader):0=3}] label_accuracy: {label_acc/(i+1):.6f}, cosine_similarity: {cosines.mean().item():.6f}, elapsed time: {second2date(time.time()-s)}', flush= True)
                s = time.time()

    # evaluate reconstruction
    results = Parallel(n_jobs= n_jobs)(delayed(reconstructMol)(smi, labels[idxs].tolist(), adj, useChiral= useChiral) for smi, idxs, adj in zip(smiles, frag_idxs_list, adjs_list))
    dec_smiles, correct = zip(*results)
    dec_smiles = list(dec_smiles)
    correct = np.array(correct)
    if useChiral:
        print(f'reconstruction-2D rate: {sum(correct > 0)/len(smiles):.6f} ({sum(correct > 0)}/{len(smiles)})', flush= True)
        print(f'reconstruction-3D rate: {sum(correct == 3)/sum(correct > 1):.6f} ({sum(correct == 3)}/{sum(correct > 1)})', flush= True)
    else:
        print(f'reconstruction rate: {sum(correct > 0)/len(smiles):.6f} ({sum(correct > 0)}/{len(smiles)})', flush= True)

    # evaluate tanimoto similarity
    tanimotos = Parallel(n_jobs= n_jobs)(delayed(calc_tanimoto)(smi1, smi2, useChiral= useChiral) for smi1, smi2 in zip(smiles, dec_smiles))
    print(f'tanimoto simirality: <mean> {np.nanmean(tanimotos):.6f}, <std> {np.nanstd(tanimotos):.6f}', flush= True)

    # evaluate distribution learning
    z_all = torch.vstack(z_list)
    print(f'z-latent similarity: <mean> {np.mean(sims_list):.6f}, <std> {np.std(sims_list):.6f}', flush= True)

    # evaluate property prediction
    if pmodel:
        ploss_list = []
        prop = torch.vstack(prop_list)
        pred = torch.vstack(pred_list)
        assert prop.shape[0] == pred.shape[0]
        prop_dim = prop.shape[-1]
        with torch.no_grad():
            for i in range(prop_dim):
                mask = ~torch.isnan(prop[:, i])
                ploss_list.append(criterion(input= pred[:, i][mask], target= prop[:, i][mask]).item())
        pred_list = pred.tolist()
        print(f'property prediction: ' + ', '.join([f'[{i}] {pl:.4f}' for i, pl in enumerate(ploss_list)]), flush= True)

    return z_all.tolist(), dec_smiles, correct, tanimotos, pred_list


def generate(dataloader,
             labels: list,
             frag_ecfps: torch.Tensor,
             ndummys: torch.Tensor,
             model: nn.Module,
             pmodel: nn.Module= None,
             max_nfrags: int= 30,
             useChiral: bool= True,
             free_n: bool= False,
             n_jobs: int= -1,
             device: torch.device= torch.device('cpu'),
             seed: int= 0
            ):
    torch_fix_seed(seed)

    labels = np.array(labels)
    z_list, pred_list, cosine_list, rmse_list = [], [], [], []
    frag_idxs_list, adjs_list = [], []
    s = time.time()
    with torch.no_grad():
        for i, data in enumerate(dataloader):
            z = data[0].to(device)

            # decode
            tree_list = model.sequential_decode(z, frag_ecfps, ndummys, max_nfrags= max_nfrags, free_n= free_n)

            # property prediction
            if pmodel:
                pred_prop = pmodel(z.to(device))
                pred_list.append(pred_prop)

            # calc similarity of latent variables
            recon_indices = get_pad_features(tree_list, key= 'fid', max_nodes_num= max_nfrags).squeeze(-1)
            features = get_pad_features(tree_list, key= 'x', max_nodes_num= max_nfrags).to(device)
            positions = get_pad_features(tree_list, key= 'pos', max_nodes_num= max_nfrags).to(device)
            src = torch.hstack([torch.full((recon_indices.shape[0], 1), -1), recon_indices.detach()]).to(device)  # for super root
            src_mask, _, src_pad_mask, _ = create_mask(src, src, pad_idx= 0, batch_first= True)
            z_dash, _, _ = model.encode(features, positions, src_mask, src_pad_mask)
            cosines = cosine_matrix(z, z_dash).diag()
            rmses = euclid_distance(z, z_dash) / np.sqrt(model.latent_dim)

            # stock in list
            z_list.append(z_dash.cpu())
            frag_idxs, adjs = zip(*[(tree.dgl_graph.ndata['fid'].squeeze(-1).tolist(), tree.adjacency_matrix().tolist()) for tree in tree_list])
            frag_idxs_list += list(frag_idxs)
            adjs_list += list(adjs)
            cosine_list += cosines.tolist()
            rmse_list += rmses.tolist()

            if ((i+1) % 10 == 0) | (i == 0) | ((i+1) == len(dataloader)):
                print(f'[{i+1:0=3}/{len(dataloader):0=3}] cosine_similarity: {cosines.mean().item():.4f}, RMSE: {rmses.mean().item():.4f}, elapsed time: {second2date(time.time()-s)}', flush= True)
                s = time.time()

    # constructMol
    dec_smiles = Parallel(n_jobs= n_jobs)(delayed(constructMol)(labels[idxs].tolist(), adj, useChiral= useChiral) for idxs, adj in zip(frag_idxs_list, adjs_list))

    z_list = torch.vstack(z_list).tolist()
    if pmodel:
        pred_list = torch.vstack(pred_list).tolist()

    # evaluate distribution learning
    print(f'z-latent similarity:', flush= True)
    print(f'- cosine:  <mean> {np.mean(cosine_list):.6f}, <std> {np.std(cosine_list):.6f}', flush= True)
    print(f'- RMSE:  <mean> {np.mean(rmse_list):.6f}, <std> {np.std(rmse_list):.6f}', flush= True)
    del z, z_dash
    torch.cuda.empty_cache()

    return z_list, dec_smiles, pred_list, cosine_list, rmse_list


def interpolate_between_2points(mol0,
                                mol1,
                                model: nn.Module,
                                labels: list,
                                frag_ecfps: torch.Tensor,
                                ndummys: torch.Tensor,
                                min_size: int= 1,
                                max_nfrags: int= 30,
                                useChiral: bool= True,
                                ignore_double: bool= True,
                                num_intp: int= 100,
                                free_n: bool= False,
                                n_jobs: int= -1,
                                device: torch.device= torch.device('cpu'),
                                ):
    # convert mol to latent variable and finger print
    tmp = []
    for mol in [mol0, mol1]:
        frags, bond_types, bondMapNums, _ = debugMolToBRICSfragments(mol, minFragSize= min_size, maxFragNums= max_nfrags,
                                                                     maxDegree= model.width, useChiral= useChiral, ignore_double= ignore_double)
        frag_idxs = [labels.index(f) for f in frags]
        frag_idxs, features, positions = get_tree_features(frag_idxs, frag_ecfps[frag_idxs], bond_types, bondMapNums, model.depth, model.width, free_n)
        frag_idxs, features, positions = frag_idxs.unsqueeze(0), features.unsqueeze(0), positions.unsqueeze(0)
        src = torch.hstack([torch.full((frag_idxs.shape[0], 1), -1), frag_idxs.detach()])
        src_mask, _, src_pad_mask, _ = create_mask(src, src, pad_idx= 0, batch_first= True)
        z, _, _ = model.encode(features.to(device), positions.to(device), src_mask.to(device), src_pad_mask.to(device))
        fp = AllChem.GetMorganFingerprintAsBitVect(mol, 2, 2048, useChirality= useChiral)
        tmp += [z, fp]
    z0, fp0, z1, fp1 = tmp

    # interpolate
    labels = np.array(labels)
    d = z1 - z0
    similarity = DataStructs.TanimotoSimilarity(fp0, fp1)
    z_intp = torch.vstack([z0 + ((i*d)/(num_intp+1)) for i in range(1, num_intp+1)]).float()

    with torch.no_grad():
        tree_list = model.sequential_decode(z_intp.to(device), frag_ecfps, ndummys, max_nfrags= max_nfrags)
    frag_idxs_list, adjs_list = map(list, zip(*[(tree.dgl_graph.ndata['fid'].squeeze(-1).tolist(), tree.adjacency_matrix().tolist()) for tree in tree_list]))
    smiles_intp = Parallel(n_jobs= n_jobs)(delayed(constructMol)(labels[idxs].tolist(), adj, useChiral= useChiral) for idxs, adj in zip(frag_idxs_list, adjs_list))
    mols_intp = [Chem.MolFromSmiles(s) for s in smiles_intp]
    similarity_intp = [DataStructs.TanimotoSimilarity(fp0, AllChem.GetMorganFingerprintAsBitVect(m, 2, 2048, useChirality= useChiral)) for m in mols_intp]

    return smiles_intp, similarity_intp, similarity


def eval_interpolate_between_2points(smiles: list,
                                     model: nn.Module,
                                     labels: list,
                                     frag_ecfps: torch.Tensor,
                                     ndummys: torch.Tensor,
                                     min_size: int= 1,
                                     max_nfrags: int= 30,
                                     useChiral: bool= True,
                                     ignore_double: bool= False,
                                     num_intp: int= 100,
                                     num_iters: int= 10000,
                                     free_n: bool= False,
                                     n_jobs: int= -1,
                                     device: torch.device= torch.device('cpu'),
                                     seed: int= 0,
                                     ):
    """
    Evaluate smoothness of latent space quantitatively
    """
    torch_fix_seed(seed)
    sample_list, unique_list, mae_list, corr_list = [], [], [], []
    s = time.time()
    for i in range(num_iters):
        random.shuffle(smiles)
        s0, s1 = smiles[:2]
        smiles_intp, similarity_intp, similarity = interpolate_between_2points(Chem.MolFromSmiles(s0), Chem.MolFromSmiles(s1), model, labels, frag_ecfps, ndummys,
                                                                               min_size, max_nfrags, useChiral, ignore_double,
                                                                               num_intp, free_n, n_jobs, device)
        d_sim = 1 - similarity
        similarity_intp = torch.tensor(similarity_intp)
        ideal_similarity_intp = torch.tensor([1 - (i*d_sim)/(num_intp+1) for i in range(1, num_intp+1)]).float()

        sample_list.append((s0, s1, similarity))
        unique_list.append(len(set([s for s in smiles_intp if s is not None]))/num_intp)
        mae_list.append(F.l1_loss(similarity_intp, ideal_similarity_intp).item())
        corr_list.append(torch.corrcoef(torch.vstack([similarity_intp, ideal_similarity_intp]))[0, 1].item())

        if ((i+1) % 100 == 0):
            print(f'[{i+1}/{num_iters}] unique: {np.mean(unique_list):.4f}, R: {np.mean(corr_list):.4f}, MAE: {np.mean(mae_list):.4f}, elapsed time: {second2date(time.time()-s)}', flush= True)
            s = time.time()

    print(f'[{i+1}/{num_iters}] unique: {np.mean(unique_list):.4f} (std; {np.std(unique_list):.4f}), R: {np.mean(corr_list):.4f} (std; {np.std(corr_list):.4f}),',
           f'MAE: {np.mean(mae_list):.4f} (std; {np.std(mae_list):.4f}), elapsed time: {second2date(time.time()-s)}', flush= True)

    return sample_list, unique_list, mae_list, corr_list


def interpolate_around(mol,
                       model: nn.Module,
                       labels: list,
                       frag_ecfps: torch.Tensor,
                       ndummys: torch.Tensor,
                       min_size: int= 1,
                       max_nfrags: int= 30,
                       useChiral: bool= True,
                       ignore_double: bool= False,
                       radius: int= 4,
                       delta: int= 5,
                       free_n: bool= False,
                       n_jobs: int= -1,
                       device: torch.device= torch.device('cpu'),
                       seed: int= 0
                        ):
    """
    num_intp_per_axis: odd number
    """
    torch_fix_seed(seed)

    # convert mol to latent variable and finger print
    frags, bond_types, bondMapNums, _ = debugMolToBRICSfragments(mol, minFragSize= min_size, maxFragNums= max_nfrags,
                                                                 maxDegree= model.width, useChiral= useChiral, ignore_double= ignore_double)
    frag_idxs = [labels.index(f) for f in frags]
    frag_idxs, features, positions = get_tree_features(frag_idxs, frag_ecfps[frag_idxs], bond_types, bondMapNums, model.depth, model.width, free_n)
    frag_idxs, features, positions = frag_idxs.unsqueeze(0), features.unsqueeze(0), positions.unsqueeze(0)
    src = torch.hstack([torch.full((frag_idxs.shape[0], 1), -1), frag_idxs.detach()])
    src_mask, _, src_pad_mask, _ = create_mask(src, src, pad_idx= 0, batch_first= True)
    z, _, _ = model.encode(features.to(device), positions.to(device), src_mask.to(device), src_pad_mask.to(device))
    z = z.cpu()
    labels = np.array(labels)
    fp = AllChem.GetMorganFingerprintAsBitVect(mol, 2, 2048, useChirality= useChiral)

    # randomly generate 2 orthonormal axis x & y.
    x = np.random.randn(z.shape[-1])
    x /= np.linalg.norm(x)

    y = np.random.randn(z.shape[-1])
    y -= y.dot(x) * x
    y /= np.linalg.norm(y)

    x, y = torch.from_numpy(x).float(), torch.from_numpy(y).float()

    z_intp = []
    for dx in range(-radius, radius + 1):
        for dy in range(-radius, radius + 1):
            z_intp.append(z + x * delta * dx + y * delta * dy)
            if (dx == 0) & (dy == 0):
                center = len(z_intp) - 1
    z_intp = torch.vstack(z_intp).float()

    with torch.no_grad():
        tree_list = model.sequential_decode(z_intp.to(device), frag_ecfps, ndummys, max_nfrags= max_nfrags)
    frag_idxs_list, adjs_list = map(list, zip(*[(tree.dgl_graph.ndata['fid'].squeeze(-1).tolist(), tree.adjacency_matrix().tolist()) for tree in tree_list]))
    smiles_intp = Parallel(n_jobs= n_jobs)(delayed(constructMol)(labels[idxs].tolist(), adj, useChiral= useChiral) for idxs, adj in zip(frag_idxs_list, adjs_list))
    mols_intp = [Chem.MolFromSmiles(s) for s in smiles_intp]
    # fp = AllChem.GetMorganFingerprintAsBitVect(mols_intp[center], 2, 2048, useChirality= useChiral)
    similarity_intp = [DataStructs.TanimotoSimilarity(fp, AllChem.GetMorganFingerprintAsBitVect(m, 2, 2048, useChirality= useChiral)) for m in mols_intp]

    return smiles_intp, similarity_intp


def eval_interpolate_around(smiles: list,
                            model: nn.Module,
                            labels: list,
                            frag_ecfps: torch.Tensor,
                            ndummys: torch.Tensor,
                            min_size: int= 1,
                            max_nfrags: int= 30,
                            useChiral: bool= True,
                            ignore_double: bool= False,
                            radius: int= 4,
                            delta: int= 5,
                            num_iters: int= 10000,
                            free_n: bool= False,
                            n_jobs: int= -1,
                            device: torch.device= torch.device('cpu'),
                            seed: int= 0,
                            ):
    """
    Evaluate smoothness of latent space quantitatively
    """
    torch_fix_seed(seed)
    sample_list, unique_dict, similarity_dict = [], {}, {}
    s = time.time()
    random.shuffle(smiles)
    for i in range(num_iters):
        s0 = smiles[i]
        sample_list.append(s0)
        for j in range(1, radius+1):
            if i == 0:
                unique_dict[f'unique:{delta*j}'] = []
                similarity_dict[f'similarity:{delta*j}'] = []
            smiles_intp, similarity_intp = interpolate_around(Chem.MolFromSmiles(s0), model, labels, frag_ecfps, ndummys,
                                                              min_size, max_nfrags, useChiral, ignore_double,
                                                              1, delta*j, free_n, n_jobs, device, seed)
            unique_dict[f'unique:{delta*j}'].append((len(set([s for s in smiles_intp if s is not None]))-1) / (len(smiles_intp)-1))
            similarity_dict[f'similarity:{delta*j}'].append((np.sum(similarity_intp)-1)/(len(similarity_intp)-1))

        if ((i+1) % 100 == 0):
            print(f'[{i+1}/{num_iters}] unique, similarity, elapsed time: {second2date(time.time()-s)}', flush= True)
            for j in range(1, radius+1):
                print(f'- delta[{delta*j}]: {np.mean(unique_dict[f"unique:{delta*j}"]):.4f}, {np.mean(similarity_dict[f"similarity:{delta*j}"]):.4f}', flush= True)
            s = time.time()

    print(f'[{i+1}/{num_iters}] unique, similarity, elapsed time: {second2date(time.time()-s)}', flush= True)
    for j in range(1, radius+1):
        print(f'- delta[{delta*j}]: {np.mean(unique_dict[f"unique:{delta*j}"]):.4f} (std; {np.std(unique_dict[f"unique:{delta*j}"]):.4f}),',
              f'{np.mean(similarity_dict[f"similarity:{delta*j}"]):.4f} (std; {np.std(similarity_dict[f"similarity:{delta*j}"]):.4f})', flush= True)

    return sample_list, unique_dict, similarity_dict


def prop_optimize(z: torch.Tensor,
                  target: torch.Tensor,
                  pmodel: nn.Module,
                  criterion: nn.Module= nn.MSELoss(reduction= 'none'),
                  lr: float= 0.01,
                  max_iter: int= 100,
                  max_patient: int= 5,
                  device: torch.device= torch.device('cpu')
                  ) -> torch.Tensor:
    """
    optimize latent-variables using gradient.
    z: torch.Tensor, shape= (batch_size, z_dim)
    target: torch.Tensor, shape= (batch_size, prop_dim)
    """
    z_opt = z.detach().clone().to(device)
    z_opt.requires_grad = True
    target = target.to(device)
    optimizer = torch.optim.Adam([z_opt], lr= lr)
    best_z, best_loss = z_opt.clone().cpu(), torch.full(size= (z.shape[0], 1), fill_value= float('inf'))

    patient = 0
    for _ in range(max_iter):
        pred = pmodel(z_opt)
        loss = criterion(pred, target)
        optimizer.zero_grad()
        pmodel.zero_grad()
        loss.mean().backward()
        optimizer.step()

        loss = loss.mean(dim= -1, keepdim= True).cpu()
        if torch.all(loss>=best_loss):
            patient += 1
        else:
            best_z = torch.where(loss<best_loss, z_opt.clone().cpu(), best_z).detach()
            best_loss = torch.where(loss<best_loss, loss, best_loss).detach()

        if patient > max_patient:
            break

    return best_z


def constrained_prop_optimize(mols: list,
                              target: torch.Tensor,
                              scoring_func,              # scoring_func(mol) -> score
                              model: nn.Module,
                              pmodel: nn.Module,
                              labels: list,
                              frag_ecfps: torch.Tensor,
                              ndummys: torch.Tensor,
                              min_size: int= 1,
                              max_nfrags: int= 30,
                              useChiral: bool= True,
                              ignore_double: bool= False,
                              criterion: nn.Module= nn.MSELoss(reduction= 'none'),
                              thresholds: list= [0.2, 0.4, 0.6],
                              lr: float= 0.01,
                              max_iter: int= 80,
                              device: torch.device= torch.device('cpu')
                              ) -> list:
    """
    Multiple properties not supported.
    Pmodel outputs have shape= (batch_size, 1)
    """
    z_all = []
    for mol in mols:
        frags, bond_types, bondMapNums, _ = debugMolToBRICSfragments(mol, minFragSize= min_size, maxFragNums= max_nfrags,
                                                                     maxDegree= model.width, useChiral= useChiral, ignore_double= ignore_double)
        frag_idxs = [labels.index(f) for f in frags]
        frag_idxs, features, positions = get_tree_features(frag_idxs, frag_ecfps[frag_idxs], bond_types, bondMapNums, model.depth, model.width, False)
        frag_idxs, features, positions = frag_idxs.unsqueeze(0), features.unsqueeze(0), positions.unsqueeze(0)
        src = torch.hstack([torch.full((frag_idxs.shape[0], 1), -1), frag_idxs.detach()])
        src_mask, _, src_pad_mask, _ = create_mask(src, src, pad_idx= 0, batch_first= True)
        z, _, _ = model.encode(features.to(device), positions.to(device), src_mask.to(device), src_pad_mask.to(device))
        z_all.append(z.cpu())
    z_all = torch.vstack(z_all)

    model.set_labels(labels)
    smiles_opt = []
    z_opt = z_all.clone()
    for _ in range(max_iter):
        z_opt = prop_optimize(z_opt, target, pmodel, criterion, lr= lr, max_iter= 1, device= device)
        with torch.no_grad():
            smis_opt = model.sequential_decode(z_opt.to(device), frag_ecfps, ndummys, max_nfrags= max_nfrags, asSmiles= True)
            smiles_opt.append(smis_opt)

    improved_smiles, improved_score = {f'thr-{thr}': [] for thr in thresholds}, {f'thr-{thr}': [] for thr in thresholds}
    for mol, smis_opt in zip(mols, zip(*smiles_opt)):
        ecfp = AllChem.GetMorganFingerprintAsBitVect(mol, 2, 2048)
        ecfps_opt = [AllChem.GetMorganFingerprintAsBitVect(Chem.MolFromSmiles(s), 2, 2048) for s in smis_opt]
        tanimotos = DataStructs.BulkTanimotoSimilarity(ecfp, ecfps_opt)
        tanimotos = np.where(tanimotos==1, 0, tanimotos)    # remove the same mols

        orig_score = scoring_func(mol)
        for thr in thresholds:
            smis_cand = [smis_opt[i] for i in range(max_iter) if tanimotos[i] >= thr]
            scores = np.array([scoring_func(Chem.MolFromSmiles(s)) for s in smis_cand])

            if np.any(scores>orig_score):
                max_improve = scores.max() - orig_score
                improved_smi = smis_cand[np.argmax(scores)]
            else:
                max_improve = float('nan')
                improved_smi = None
            improved_smiles[f'thr-{thr}'].append(improved_smi)
            improved_score[f'thr-{thr}'].append(max_improve)

    return improved_smiles, improved_score


def featurelize_fragments(dataloader,
                          model: nn.Module,
                          frag_ecfps: torch.Tensor,
                          root: bool= False,
                          device: torch.device= torch.device('cpu')
                          ):
    attn_dict = {i: [] for i in range(len(frag_ecfps))}
    start = time.time()
    with torch.no_grad():
        for iter, data in enumerate(dataloader):
            frag_indices = data[0]
            features = frag_ecfps[frag_indices.flatten()].reshape(frag_indices.shape[0], frag_indices.shape[1], -1).to(device)
            positions = data[1].to(device)

            # make mask
            src = torch.hstack([torch.full((frag_indices.shape[0], 1), -1), frag_indices.detach()]).to(device)  # for super root
            src_mask, _, src_pad_mask, _ = create_mask(src, src, pad_idx= 0, batch_first= True)

            # positional embbeding
            src = model.fc_ecfp(features) + model.PE(positions)           # (B, L, d_model)

            # attach super root
            root_embed = model.embed(src.new_zeros(src.shape[0], 1).long())
            src = torch.cat([root_embed, src], dim= 1)                    # (B, L+1, d_model)

            x = src
            attn_values = []
            for layer in model.encoder.layers:
                attn, attn_map = layer.self_attn(x, x, x, attn_mask= src_mask, key_padding_mask= src_pad_mask, need_weights= True)
                x = layer(x, src_mask, src_pad_mask)
                attn_value = attn_map[:, 0, :] if root else attn_map.mean(dim= 1)  # if True, use only root node values
                attn_values.append(attn_value.unsqueeze(-1))
            attn_values = torch.stack(attn_values, dim= -1)                  # (B, L+1, len(layers))

            for idxs, attn in zip(frag_indices, attn_values):
                for i, values in enumerate(attn[1:]):
                    attn_dict[idxs[i].item()].append(values.squeeze(0).tolist())

            if (iter==0) | (((iter+1) % 10) == 0) | (iter==len(dataloader)-1):
                print(f'[{iter+1}/{len(dataloader)}] elapsed time: {second2date(time.time()-start)}', flush= True)
                start = time.time()

    attn_mean, attn_std = [], []
    for key, values in attn_dict.items():
        values = np.array(values)
        if len(values) == 0:
            mu = [0] * len(model.encoder.layers)
            std = [0] * len(model.encoder.layers)
        else:
            mu = values.mean(axis= 0).tolist()
            std = values.std(axis= 0).tolist()
        attn_mean.append(mu)
        attn_std.append(std)
    del attn_dict

    return attn_mean, attn_std


if __name__ == '__main__':
    pmodel = nn.Linear(128, 2)
    z = torch.rand(6, 128)
    target = torch.ones(6, 2)

    best_z = prop_optimize(z, target, pmodel)
    print(best_z)

tensor([[ 2.0061e-01,  2.8346e-01,  6.2736e-01,  6.3229e-01, -3.1397e-02,
          4.6346e-01,  5.6554e-01,  8.3171e-01,  3.7000e-01, -9.8776e-02,
         -2.6858e-01,  1.2034e+00,  4.8364e-01,  3.3301e-01,  7.2608e-02,
          4.6348e-01,  2.0385e-02,  3.3629e-01,  8.7015e-03,  6.8050e-01,
         -2.4289e-01,  1.0903e+00, -1.4920e-01,  6.2988e-02,  1.1061e+00,
          4.0537e-01,  8.9739e-01,  4.3760e-01,  5.0841e-02,  5.9734e-01,
          8.6879e-01,  4.0403e-01,  7.2367e-01,  9.6358e-01,  5.0858e-01,
          6.1279e-01,  5.6733e-01,  5.6515e-01,  1.0285e+00,  5.4998e-01,
          5.7119e-01,  8.8476e-01, -2.2863e-01,  3.4330e-01,  4.6614e-02,
         -1.2995e-01,  7.6173e-01,  4.9361e-01,  5.8316e-01,  9.3524e-01,
          3.9067e-01,  1.2880e+00,  6.1557e-01,  4.1987e-01,  4.3492e-03,
          4.6465e-01,  5.0646e-01,  9.4691e-01,  8.4681e-01, -2.7711e-01,
          6.4369e-03,  1.0989e+00,  5.9787e-01,  1.8777e-02,  2.8824e-01,
          6.0414e-01,  5.7607e-01,  1.

In [ ]:
# type(uni_fragments)

# From here, our original codes appear

## "Generate" function for MOBO

In [ ]:
import torch
import numpy as np
from torch import nn
from joblib import Parallel, delayed
import time

def generate_for_bo(
        z_candidates: torch.Tensor,
        labels: np.ndarray,
        frag_ecfps: torch.Tensor,
        ndummys: torch.Tensor,
        model: nn.Module,
        max_nfrags: int = 30,
        useChiral: bool = True,
        n_jobs: int = -1,
        device: torch.device = torch.device("cpu"),
        seed: int = 0
    ):

    torch.manual_seed(seed)
    np.random.seed(seed)
    labels = np.array(labels)
    z_candidates = z_candidates.to(device)
    model.eval()
    if pmodel:
        pmodel.eval()

    dec_smiles, pred_list, valid_mask = [], [], []

    with torch.no_grad():
        # decode latent -> molecular tree
        tree_list = model.sequential_decode(
            z_candidates,
            frag_ecfps,
            ndummys,
            max_nfrags=max_nfrags,
            free_n=False
        )

        # property prediction
        if pmodel:
            pred_tensor = pmodel(z_candidates)
            pred_list = pred_tensor.cpu().numpy().tolist()
        else:
            pred_list = None

        # constructMol: fragment IDs + adjacency -> SMILES
        frag_idxs_list, adjs_list = zip(*[
            (
                tree.dgl_graph.ndata["fid"].squeeze(-1).tolist(),
                tree.adjacency_matrix().tolist()
            ) for tree in tree_list
        ])

        smiles_out = Parallel(n_jobs=n_jobs)(
            delayed(constructMol)(
                # labels[idxs].tolist(),  # fragment labels
                labels[idxs],  # fragment labels
                adj,
                useChiral=useChiral
            )
            for idxs, adj in zip(frag_idxs_list, adjs_list)
        )
        dec_smiles = Parallel(n_jobs= n_jobs)(
            delayed(constructMol)(
                # labels[idxs].tolist(),
                labels[idxs],
                adj,
                useChiral= useChiral
                )
            for idxs, adj in zip(frag_idxs_list, adjs_list))
        for smi in smiles_out:
            if smi is None:
                dec_smiles.append(None)
                valid_mask.append(False)
            else:
                # dec_smiles.append(smi)
                valid_mask.append(True)

    torch.cuda.empty_cache()
    print("dec_smiles: ", dec_smiles)
    # print("smiles_out: ", smiles_out)
    return dec_smiles, pred_list


## changing directory

In [ ]:
experiment_number = "try_it_to_valify"

In [ ]:
import os
path=rf"/content/drive/{experiment_number}/{experiment_number}_logs"

if os.path.exists(path)==False:
  os.makedirs(path)
  os.chdir(path)
else:
  os.chdir(path)

print(os.getcwd())

/content/drive/try_it_to_valify/try_it_to_valify_logs


## preparation to call generative models

In [ ]:
import argparse
import os

# set parser
parser = argparse.ArgumentParser(description='please enter paths')
parser.add_argument('--yml', type=str, help='yml file')
parser.add_argument('--load_epoch', type=int, default=None)
parser.add_argument('--max_nfrags', type=int, default=None)
parser.add_argument('--N', type=int, default=5)
parser.add_argument('--k', type=int, default=10000)
parser.add_argument('--t', type=float, default=1.0)
parser.add_argument('--gpu', type=int, default=0)
parser.add_argument('--n_jobs', type=int, default=1)
parser.add_argument('--free_n', action='store_true')
parser.add_argument('--recon', action='store_true', help='reconstruction mode', default=False)
parser.add_argument('--gen', action='store_true', help='generation mode', default=True)


args, _ = parser.parse_known_args([
    "--yml", "/content/extracted/Polymer_standardized_struct/input_data/params.yml",

    "--N", "5",
    "--k", "10000",
    "--gpu", "0",
    "--n_jobs", "24"
])
import datetime
import gc
import pickle
import sys
import time
import warnings
from joblib import Parallel, delayed
warnings.simplefilter('ignore')

import numpy as np
import pandas as pd
import yaml
# import moses
from rdkit import Chem, RDLogger
RDLogger.DisableLog('rdApp.*')
lg = RDLogger.logger()
lg.setLevel(RDLogger.CRITICAL)

import torch
from torch.utils.data import DataLoader, TensorDataset
from torch.distributions.multivariate_normal import MultivariateNormal



if args.recon == args.gen:
    args.recon = args.gen = True

yml_file = args.yml

start = time.time()
print(f'---{datetime.datetime.now()}: start.---', flush= True)

## check environments
if torch.cuda.is_available():
    device = 'cuda'
    torch.cuda.set_device(args.gpu)
else:
    device = 'cpu'
print(f'GPU [{args.gpu}] is available: {torch.cuda.is_available()}\n', flush= True)

## load hyperparameters
with open(yml_file) as yml:
    params = yaml.safe_load(yml)
print(f'load: {yml_file}', flush= True)

# # path

result_path= # directory where you want to save your results
data_path = "/results/data/Polymer_standardized_struct.csv"
frag_path = "/content/extracted/Polymer_standardized_struct/input_data/fragments.csv"


# hyperparameters for decomposition and tree-fragments
decomp_params = params['decomp']
n_bits = decomp_params['n_bits']
max_nfrags = decomp_params['max_nfrags'] if args.max_nfrags is None else args.max_nfrags
dupl_bits = decomp_params['dupl_bits']
radius = decomp_params['radius']
max_depth = decomp_params['max_depth']
max_degree = decomp_params['max_degree']
useChiral = decomp_params['useChiral']
ignore_double = decomp_params['ignore_double']
ignore_dummy = decomp_params['ignore_dummy']

# hyperparameters for model
model_params = params['model']
d_model = model_params['d_model']
d_ff = model_params['d_ff']
num_layers = model_params['nlayer']
num_heads = model_params['nhead']
activation = model_params['activation']
latent_dim = model_params['latent']
feat_dim = model_params['feat']
props = model_params['property']
pnames = list(props.keys())
ploss = model_params['ploss']

# hyperparameters for training
train_params = params['train']
batch_size = train_params['batch_size']
# batch_size = 128

## load data
df_frag = pd.read_csv(frag_path)
uni_fragments = df_frag['SMILES'].tolist()
freq_list = df_frag['frequency'].tolist()
try:
    with open(os.path.join(result_path, 'input_data', '/content/extracted/Polymer_standardized_struct/input_data/csr_ecfps.pkl'), 'rb') as f:

        frag_ecfps = pickle.load(f).toarray()
        frag_ecfps = torch.from_numpy(frag_ecfps).float()
    assert frag_ecfps.shape[0] == len(uni_fragments)
    assert frag_ecfps.shape[1] == (n_bits + dupl_bits)
except Exception as e:
    print(e, flush= True)
    frag_ecfps = torch.tensor(SmilesToMorganFingetPrints(uni_fragments[1:], n_bits= n_bits, dupl_bits= dupl_bits, radius= radius,
                                                        ignore_dummy= ignore_dummy, useChiral= useChiral, n_jobs= args.n_jobs)).float()
    frag_ecfps = torch.vstack([frag_ecfps.new_zeros(1, n_bits+dupl_bits), frag_ecfps])      # padding feature is zero vector
ndummys = torch.tensor(df_frag['ndummys'].tolist()).long()
prop_dim = sum(list(props.values())) if pnames else None
print(f'data: {data_path}, useChiral: {useChiral}, n_jobs: {args.n_jobs}', flush= True)
print(f'fragments: {len(uni_fragments)}, feature: {frag_ecfps.shape[-1]}, tree: ({max_depth}, {max_degree}), prop: {prop_dim}', flush= True)

# load model
num_labels = frag_ecfps.shape[0]
if prop_dim:
    pmodel = propLinear(latent_dim, prop_dim).to(device)
    if args.load_epoch:
        pmodel.load_state_dict(torch.load(os.path.join(result_path, 'models', f'pmodel_iter{args.load_epoch}.pth'), map_location= device))
    else:
        pmodel.load_state_dict(torch.load(os.path.join(result_path, 'models', f'pmodel_best.pth'), map_location= device))

    pmodel.eval()
else:
    pmodel = None
model = FRATTVAE(num_labels, max_depth, max_degree, feat_dim, latent_dim,
               d_model, d_ff, num_layers, num_heads, activation).to(device)

---2026-01-30 10:27:55.555487: start.---
GPU [0] is available: True

load: /content/extracted/Polymer_standardized_struct/input_data/params.yml
data: /results/data/Polymer_standardized_struct.csv, useChiral: False, n_jobs: 24
fragments: 1955, feature: 2048, tree: (32, 8), prop: None


### $n_{D}$

#### our original predictor  $\mathit{N} \alpha$

#### transplanted

In [ ]:

from rdkit.Chem import AllChem
from rdkit.Chem import Crippen
from pickle import load
import numpy as np
from sklearn.linear_model import  Ridge, Lasso
from rdkit import RDLogger
import datetime
from rdkit import Chem
from IPython.display import HTML
import pandas as pd
import numpy as np
from rdkit import rdBase, Chem, DataStructs
from rdkit.Chem import AllChem, Draw, Descriptors, PandasTools
from rdkit.ML.Descriptors import MoleculeDescriptors
import rdkit

Failed to find the pandas get_adjustment() function to patch
Failed to patch pandas - PandasTools will have limited functionality


In [ ]:
import statsmodels.api as sm
def N_predictor(SMILES, path_model_alpha, path_model_V, constant):


    loaded_model_V = sm.load(path_model_V)

    loaded_model_alpha = sm.load(path_model_alpha)



    Volume_list_test=[]

    false_list_test=[]

    data_frame_trying=pd.DataFrame([])
    data_frame_smiles=pd.DataFrame([SMILES], columns=["SMILES"])
    data_frame_trying=pd.concat([data_frame_trying, data_frame_smiles], axis=1)
    smiles=data_frame_trying["SMILES"].to_list()[0]
    try:
        params = Chem.SmilesParserParams()
        params.removeHs = False
        m = Chem.MolFromSmiles(smiles,params)
        mol= Chem.AddHs(m)
        AllChem.EmbedMolecule(mol,useRandomCoords=True)
        Volume_list_test.append(AllChem.ComputeMolVolume(mol))

    except:
        try:
            params = Chem.SmilesParserParams()
            params.removeHs = False
            m = Chem.MolFromSmiles(smiles,params)
            mol= Chem.AddHs(m)
            AllChem.EmbedMolecule(mol,maxAttempts=5000)
            Volume_list_test.append(AllChem.ComputeMolVolume(mol))
        except RuntimeError:
            false_list_test.append(smiles)
    print("Volume_list_test", Volume_list_test)
    print("false_list_test", false_list_test)

    df_volumelist_test=pd.DataFrame(Volume_list_test, columns=["rdkit-Calculated_Volume"])
    data_frame_trying=pd.concat([data_frame_trying, df_volumelist_test], axis=1)

    PandasTools.AddMoleculeColumnToFrame(data_frame_trying,'SMILES','Molecule',includeFingerprints=True)
    data_frame_trying.head()


    descriptors_list2=pd.DataFrame(Descriptors._descList)
    a1=descriptors_list2[0]
    calc = MoleculeDescriptors.MolecularDescriptorCalculator(a1)
    descs = [calc.CalcDescriptors(mol) for mol in data_frame_trying["Molecule"]]

    df_deicriptors=pd.DataFrame(descs)
    df_deicriptors.head()

    # for i in range(0, 210, 1):
    for i in range(len(a1)):
        df_deicriptors=df_deicriptors.rename(columns={i:a1[i]})
        # print(a1[i])
    df_deicriptors.head()


    data_frame_trying=pd.concat([data_frame_trying, df_deicriptors], axis=1)
    df = data_frame_trying.copy()
    new_column_train = pd.DataFrame(df["fr_unbrch_alkane"]*df["rdkit-Calculated_Volume"], columns=["unbrch-Vol"])
    df = pd.concat([df, new_column_train], axis=1)

    adopt_list_alpha =['NumSaturatedRings',
                      'MolMR',
                      'fr_NH0',
                      'fr_NH1',
                      'fr_NH2',
                      'fr_allylic_oxid',
                      'fr_ester',
                      'fr_ketone',
                      'fr_methoxy',
                      'fr_sulfone',
                      'unbrch-Vol'
                  ]
    adopt_list_V = ['fr_C_O_noCOO',
                     'fr_allylic_oxid',
                     'fr_benzene',
                     'fr_ester',
                     'fr_sulfone',
                     'rdkit-Calculated_Volume',
                     'unbrch-Vol'
                   ]
    df_alpha =df[adopt_list_alpha]
    df_V = df[adopt_list_V]
    df_alpha =sm.add_constant(df_alpha, has_constant='add')
    df_V = sm.add_constant(df_V, has_constant='add')



    predictions_alpha = loaded_model_alpha.predict(df_alpha)
    print(predictions_alpha)

    predictions_V = loaded_model_V.predict(df_V)
    print(predictions_V)



    alpha_over_V_test=predictions_alpha/predictions_V*constant

    n_predcited_test=np.nan_to_num(np.sqrt((1+2*alpha_over_V_test)/(1-alpha_over_V_test)), nan=0)

    return n_predcited_test

In [ ]:
import os
os.getcwd()

'/content/drive/try_it_to_valify/try_it_to_valify_logs'

In [ ]:
nD = N_predictor("C1CCCCC1",
                "/content/model_alpha_VIF_2025-10-13.pkl",
                "/content/model_V_VIF_2025-10-13.pkl",

            23)
nD

Failed to patch pandas - unable to change molecule rendering


Volume_list_test [100.48000000000002]
false_list_test []
0    1.222457
dtype: float64
0    89.847624
dtype: float64


array([1.53831161])

In [ ]:
import statsmodels
statsmodels.__version__

'0.14.5'

In [ ]:
import numpy as np
np.__version__

'1.26.4'

In [ ]:
import pandas as pd
pd.__version__

'2.2.2'

#### new_Nalpha_calcrator

In [ ]:
def Nalpha_calcurator(smiles_list, path_model_alpha, path_model_V, constant):

    i=0
    filtered_smiles_list, fail_idx_list, fail_smiles_list \
        = filter_valid_with_timeout(smiles_list)
    print("fail_smiles_list_Nalpha",  fail_smiles_list)

    list_nD=[]
    if not filtered_smiles_list:
        return -30.0 * torch.ones(len(smiles_list))
    for i in range(len(filtered_smiles_list)):
        smiles_chosen = filtered_smiles_list[i]
        if i in fail_idx_list:
            nD = -30*np.ones(1)
            nD_predicted = nD
        else:
            nD_predicted=N_predictor(smiles_chosen, path_model_alpha, path_model_V, constant)


        X=nD_predicted
        list_nD.append(X)
    _Nalpha_tensor=torch.from_numpy(np.array(list_nD))

    Nalpha_tensor = _Nalpha_tensor

    return Nalpha_tensor

### HOMO-LUMO calculation by gpu4pyscf

In [ ]:
import os
os.getcwd()

'/content/drive/try_it_to_valify/try_it_to_valify_logs'

#### HOMO_LUMO_Gap_predictor(SMILES) gpu4pyscf

In [ ]:
import time
from pyscf import gto, scf
from gpu4pyscf import scf as gpu_scf
from rdkit import Chem
from rdkit.Chem import AllChem
from pyscf import gto
from gpu4pyscf import dft
from pyscf.geomopt import geometric_solver
# import pubchempy as pcp
def HOMO_LUMO_predictor_GPU4pyscf(smiles):
    # --- SMILES to #D structure ---
    mol = Chem.MolFromSmiles(smiles)
    mol = Chem.AddHs(mol)
    AllChem.EmbedMolecule(mol, randomSeed=1)
    AllChem.MMFFOptimizeMolecule(mol)

    # --- XYZ ---
    xyz_block = Chem.MolToXYZBlock(mol).splitlines()[2:]
    xyz_str = "\n".join(xyz_block)

    print("使用するXYZデータ:\n", xyz_str)

    # --- PySCF分子を構築 ---
    mol_pyscf = gto.M(
        atom=xyz_str,   # ←ここが重要：ヘッダー除去済み
        basis="sto-3g"
    )
    start_gpu = time.time()
    mf = dft.RKS(mol_pyscf, xc='B3LYP').density_fit().to_gpu()
    mf.chkfile = path + '.chk'
    mf.verbose = 1
    energies = [] # エネルギー記録用
    geometries = [] # 座標記録用
    def cb(envs): mf = envs['g_scanner'].base # 現在の計算オブジェクトを取得 energies.append(mf.e_tot) # エネルギーをリストに追加 geometries.append(mol.atom_coords(unit="ANG")) # ジオメトリーをリストに追加
    conv_params = { 'convergence_energy': 1e-5, 'convergence_grms': 3e-4, 'convergence_gmax': 4.5e-4, 'convergence_drms': 1.2e-3, 'convergence_dmax': 1.8e-3
    }
    # 構造最適化実行

    geometric_solver.optimize(mf, callback=cb, **conv_params)
    # GPU計算
    end_gpu = time.time()
    gpu_time = end_gpu - start_gpu

    print(f"GPU計算時間: {gpu_time:.2f}秒")
    # 最適化後のジオメトリーを取得
    final_geometry = mol_pyscf.atom_coords(unit="ANG")
    # SCF計算の再実行
    mf = dft.RKS(mol_pyscf)
    mf.chkfile = 'checkpoint_file.chk'
    mf.xc = 'B3LYP'
    mf.kernel()
    # 計算結果の検証
    if mf.mo_occ is not None and len(mf.mo_occ) > 0:
        print("SCF calculation completed successfully.")
        print(f"Number of occupied orbitals: {sum(mf.mo_occ > 0)}")
    else:
        raise ValueError( "SCF calculation failed or mo_occ is not properly assigned." )
    # HOMOおよびLUMOのインデックスを取得
    homo_index = (mf.mo_occ > 0).nonzero()[0][-1]
    lumo_index = (mf.mo_occ == 0).nonzero()[0][0]
    hartree_to_ev = 27.2114
    homo = mf.mo_energy[homo_index]*hartree_to_ev
    lumo = mf.mo_energy[lumo_index]*hartree_to_ev
    gap = lumo-homo
    gap = gap.get()
    gap = torch.from_numpy(gap)
    return gap

In [ ]:
def GAP_calcurator(smiles_list):

    filtered_smiles_list, fail_idx_list, fail_smiles_list \
        = filter_valid_with_timeout(smiles_list)
    print("fail_smiles_list",  fail_smiles_list)



    list_GAP=[]
    if not filtered_smiles_list:
        print("len(filtered_smiles_list): ", len(filtered_smiles_list))
        return -30.0 * torch.ones(len(smiles_list))

    for number_chosen in range(len(smiles_list)):
        if number_chosen in fail_idx_list:

            GAP = -30*np.ones(1)
        else:

            smiles_chosen = filtered_smiles_list[number_chosen]

            print("smiles_chosen: ", smiles_chosen)

            GAP=HOMO_LUMO_predictor_GPU4pyscf(smiles_chosen).detach().numpy()
            GAP = GAP*np.ones(1)

        print("GAP: ", GAP)
        print("GAP type: ", type(GAP))
        list_GAP.append(GAP)
    print("list_GAP: ", list_GAP)
    _GAP_tensor=torch.from_numpy(np.array(list_GAP))
    GAP_tensor = _GAP_tensor


    print("_GAP_tensor: ", _GAP_tensor)


    return GAP_tensor


## obj_func

In [ ]:
import os
os.getcwd()

'/content/drive/try_it_to_valify/try_it_to_valify_logs'

#### obj_func

In [ ]:

def obj_func(z, vae, q):

    z = z.to(torch.float32)


    z = z.detach().cpu()[:q, :256].cuda()


    results_picker = []

    result_collector = torch.tensor([]).to("cuda")
    smiles_list = []
    # for i in z.size()[0]:
    for i in range(q):
        # print(f"{i} of {q} (q = {q})in the roop has started generation")
        z_chosen = z[i:i+1, :256]
        # print("z_chosen:  ", z_chosen)
        dataloader = DataLoader(
            TensorDataset(z_chosen), batch_size=batch_size, shuffle=False
        )

        dec_smiles, pred_list = generate_for_bo(
            z_chosen,
            labels=uni_fragments,
            frag_ecfps=frag_ecfps,
            ndummys=torch.tensor(df_frag["ndummys"].tolist()).long(),
            model=model,
            max_nfrags=30,
            useChiral=True,
            n_jobs=-1,
            device=torch.device("cuda")
            if torch.cuda.is_available()
            else torch.device("cpu"),
            seed=0,
        )


        smiles_list.append(dec_smiles)

    Nalpha = Nalpha_calcurator(smiles_list,
                               "/content/model_alpha_VIF_2025-10-13.pkl",
                               "/content/model_V_VIF_2025-10-13.pkl",
                                2.3e1).double().squeeze(-1)

    GAP = GAP_calcurator(smiles_list).double()

    for j in range(q):
        Nalpha_chosen = Nalpha[j]
        Nalpha_chosen = Nalpha_chosen.reshape(1, 1)

        GAP_chosen = GAP[j]
        GAP_chosen = GAP_chosen.reshape(1, 1)

        result_tensor_concatnated = torch.cat([Nalpha_chosen, GAP_chosen], dim=-1).double().to("cuda")
        if j== 0:
            result_collector = result_tensor_concatnated.double().to("cuda")
        else:
            result_collector = torch.cat([result_collector, result_tensor_concatnated], dim=-1).double().to("cuda")
    result_collector = result_collector.double().to("cuda")


    if torch.cuda.is_available:

        return result_collector.double().to("cuda"), smiles_list
    else:
        return result_collector.double(), smiles_list

## MOBO roupes implemented by BoTorch

In [ ]:
import gzip
import pickle
import pandas as pd
import torch
import math
from botorch.optim import optimize_acqf
from botorch.acquisition import UpperConfidenceBound
# from botorch.models import SingleTaskGP
from botorch.fit import fit_gpytorch_mll
from botorch.utils.transforms import (standardize,
                                      normalize,
                                      unnormalize)
from gpytorch.mlls import ExactMarginalLogLikelihood


from botorch.utils.multi_objective.box_decompositions import NondominatedPartitioning
from botorch.acquisition.multi_objective.logei import qLogNoisyExpectedHypervolumeImprovement
from botorch.utils.multi_objective.pareto import is_non_dominated
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch.models import SingleTaskGP, ModelListGP


from torch.utils.data import DataLoader, TensorDataset


from rdkit.Chem import AllChem
from rdkit.Chem import Crippen
from pickle import load
import numpy as np
from sklearn.linear_model import  Ridge, Lasso
from rdkit import RDLogger
import datetime
from rdkit import Chem
from IPython.display import HTML
import pandas as pd
import numpy as np
from rdkit import rdBase, Chem, DataStructs
from rdkit.Chem import AllChem, Draw, Descriptors, PandasTools
from rdkit.ML.Descriptors import MoleculeDescriptors
import rdkit

#### preparation of repeating_detections

In [ ]:
from rdkit import Chem
import networkx as nx
from networkx.algorithms import isomorphism
from collections import defaultdict

def mol_to_nx(mol):
    """RDKit Mol -> NetworkX Graph"""
    G = nx.Graph()
    for atom in mol.GetAtoms():
        idx = atom.GetIdx()
        G.add_node(idx,
                   atom_symbol=atom.GetSymbol(),
                   aromatic=atom.GetIsAromatic(),
                   atomic_num=atom.GetAtomicNum())
    for bond in mol.GetBonds():
        a, b = bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()
        btype = str(bond.GetBondType())
        G.add_edge(a, b, bond_type=btype)
    return G

def get_ring_systems(mol):
    """
    With the help of ChatGPT
    """
    ri = mol.GetRingInfo()
    atom_rings = [set(r) for r in ri.AtomRings()]
    ring_graph = nx.Graph()
    for i in range(len(atom_rings)):
        ring_graph.add_node(i)
    for i in range(len(atom_rings)):
        for j in range(i + 1, len(atom_rings)):
            if atom_rings[i] & atom_rings[j]:  # 原子を共有しているなら融合
                ring_graph.add_edge(i, j)
    # 連結成分ごとに fused ring system をまとめる
    ring_systems = []
    for comp in nx.connected_components(ring_graph):
        atoms_in_system = set().union(*[atom_rings[i] for i in comp])
        ring_systems.append(sorted(list(atoms_in_system)))
    return ring_systems

def wl_hash_subgraph(G, node_indices):
    subG = G.subgraph(node_indices).copy()
    try:
        h = nx.weisfeiler_lehman_graph_hash(subG, node_attr='atom_symbol', edge_attr='bond_type')
    except TypeError:
        h = nx.weisfeiler_lehman_graph_hash(subG)
    return h, subG

def is_isomorphic(G1, G2):
    nm = isomorphism.categorical_node_match(['atom_symbol', 'aromatic'], [None, None])
    em = isomorphism.categorical_edge_match('bond_type', None)
    GM = isomorphism.GraphMatcher(G1, G2, node_match=nm, edge_match=em)
    return GM.is_isomorphic()


def subgraph_to_smiles(mol, atom_indices):
    atom_set = set(atom_indices)
    emol = Chem.EditableMol(Chem.Mol())


    atom_map = {}
    for old_idx in atom_indices:
        old_atom = mol.GetAtomWithIdx(old_idx)
        new_atom = Chem.Atom(old_atom.GetSymbol())
        new_atom.SetIsAromatic(old_atom.GetIsAromatic())
        new_idx = emol.AddAtom(new_atom)
        atom_map[old_idx] = new_idx

    # Add bonding if needed
    for bond in mol.GetBonds():
        a1, a2 = bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()
        if a1 in atom_set and a2 in atom_set:
            emol.AddBond(atom_map[a1], atom_map[a2], bond.GetBondType())

    submol = emol.GetMol()
    Chem.SanitizeMol(submol)

    # canonical SMILES to close the rings
    smi = Chem.MolToSmiles(submol, canonical=True)
    return smi

def extract_fused_ring_systems_with_smiles(smiles, verbose=True):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        raise ValueError("Invalid SMILES")

    G = mol_to_nx(mol)
    ring_systems = get_ring_systems(mol)
    if verbose:
        print(f"Found {len(ring_systems)} ring systems")

    groups = defaultdict(list)
    cache = {}
    for rs_atoms in ring_systems:
        h, subG = wl_hash_subgraph(G, rs_atoms)
        cache[h] = subG
        groups[h].append(rs_atoms)

    motifs = []
    for h, ring_lists in groups.items():
        buckets = []
        for atoms in ring_lists:
            sg = G.subgraph(atoms).copy()
            placed = False
            for b in buckets:
                rep_atoms = b[0]
                rep_sg = G.subgraph(rep_atoms).copy()
                if is_isomorphic(rep_sg, sg):
                    b.append(atoms)
                    placed = True
                    break
            if not placed:
                buckets.append([atoms])

        for b in buckets:
            rep_atoms = b[0]
            rep_smi = subgraph_to_smiles(mol, rep_atoms)
            motifs.append({
                "count": len(b),
                "atom_indices": b,
                "smiles": rep_smi,
                "ring_size": len(rep_atoms),
            })

    if verbose:
        for m in motifs:
            print(f"Motif: {m['smiles']}  (count={m['count']}, atoms={m['atom_indices'][0]})")
    return motifs



#### filter_valid sanitizing generated molecule

In [ ]:
import gzip
import math
import pickle
from tqdm import tqdm
from rdkit import Chem
from rdkit.Chem.Descriptors import ExactMolWt
import torch



def trying_roop_of_filter_valid(each_smiles):
    success_smiles = []
    # fail_idx_ = []
    fail_smiles = []
    try:
        smiles = Chem.MolToSmiles(Chem.MolFromSmiles(each_smiles))
        Mn=ExactMolWt(Chem.MolFromSmiles(smiles))
        if Mn<100:
            # Delete error-causing molecule beforehand
            mol = Chem.AddHs(Chem.MolFromSmiles(smiles))
            target_atom_symbol = "Si"  # The symbol of the atom you are searching for
            found_atoms = []

            for atom in mol.GetAtoms():
                if atom.GetSymbol() == target_atom_symbol:
                    found_atoms.append(atom)
                    # fail_idx_list.append(each_idx)
                    fail_smiles.append(each_smiles)
                    success_smiles.append("C")
                    continue

            if not found_atoms:
                # print(f"No {target_atom_symbol} atoms found in the SMILES string.")
                ring_info = mol.GetRingInfo()
                # counting ring number
                num_atoms = mol.GetNumAtoms()

                list_num_rings=[]
                for i in range(num_atoms):
                    for j in enumerate(ring_info.AtomRingSizes(i)):
                        # print(j[1])
                        list_num_rings.append(j[1])
                if list_num_rings ==[]:
                    success_smiles.append(smiles)
                else:
                    max_ring_size=max(list_num_rings)

                    if max_ring_size<8:
                        AllChem.EmbedMolecule(mol)
                        AllChem.ComputeMolVolume(mol)

                        success_smiles.append(smiles)

        else: #Mn>1000
            motifs = extract_fused_ring_systems_with_smiles(smiles)
            smiles_fragments = motifs[0]["smiles"]
            print("smiles_fragments: ", smiles_fragments)
            mol = Chem.AddHs(Chem.MolFromSmiles(smiles_fragments))
            found_atoms = []

            for atom in mol.GetAtoms():

                target_atom_symbol = "Si"
                if atom.GetSymbol() == target_atom_symbol:
                    found_atoms.append(atom)
                    # fail_idx_list.append(each_idx)
                    fail_smiles.append(smiles_fragments)
                    success_smiles.append("C")
                    continue
            if not found_atoms:

                ring_info = mol.GetRingInfo()

                num_atoms = mol.GetNumAtoms()

                list_num_rings=[]
                for i in range(num_atoms):
                    for j in enumerate(ring_info.AtomRingSizes(i)):

                        list_num_rings.append(j[1])
                if list_num_rings ==[]:
                    success_smiles.append(smiles_fragments)
                else:
                    max_ring_size=max(list_num_rings)

                    if max_ring_size<8:
                        AllChem.EmbedMolecule(mol)
                        AllChem.ComputeMolVolume(mol)

                        success_smiles.append(smiles_fragments)






    except:
        success_smiles.append("C")
        fail_smiles.append(each_smiles)
    return success_smiles, fail_smiles

def filter_valid(smiles_list):
    print("before_filter: ", smiles_list)
    success_list = []
    fail_idx_list = []
    fail_smiles_list = []
    if len(smiles_list)==1:
        each_idx = 0
        each_smiles = smiles_list[0][0] if type(smiles_list[0][0]) == "str" else  smiles_list[0][0][0]
        success_list, fail_smiles_list = trying_roop_of_filter_valid(each_smiles)

    else:
        for each_idx, each_smiles in enumerate(smiles_list):

            success_list_sub, fail_smiles_list_sub = trying_roop_of_filter_valid(each_smiles[0])
            if success_list_sub == []:
                fail_idx_list.append(each_idx)
                fail_smiles_list.append(each_smiles[0])

            else:

              success_list.append(success_list_sub[0])

    print("success_list: ", success_list)
    return success_list, fail_idx_list,fail_smiles_list

In [ ]:
from timeout_decorator import timeout, TimeoutError

@timeout_decorator.timeout(60)
def filter_valid_with_timeout(smiles_list):
    try:
        success_list, fail_idx_list,fail_smiles_list = filter_valid(smiles_list)
        return success_list, fail_idx_list,fail_smiles_list
    except timeout_decorator.timeout_decorator.TimeoutError:
        return list(["C", "C","C","C","C","C","C","C",]), np.arange(0, len(smiles_list), 1).tolist(), smiles_list



In [ ]:
model.cuda()

FRATTVAE(
  (embed): Embedding(1, 512)
  (fc_ecfp): Sequential(
    (0): Linear(in_features=2048, out_features=1024, bias=True)
    (1): Linear(in_features=1024, out_features=512, bias=True)
  )
  (PE): TreePositionalEncoding(
    (fc): Linear(in_features=512, out_features=512, bias=True)
  )
  (encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-5): 6 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=512, out_features=512, bias=True)
        )
        (linear1): Linear(in_features=512, out_features=2048, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=2048, out_features=512, bias=True)
        (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )

####  distance_constraint

In [ ]:
import torch
torch.manual_seed(0)
def distance_constraint(X):
    eps = 0.1
    return torch.min(torch.norm(X - 1, dim=-1)) - eps

#### optimization

In [ ]:
# import torch
from botorch.models import SingleTaskGP, ModelListGP
from gpytorch.mlls.sum_marginal_log_likelihood import SumMarginalLogLikelihood
from botorch.sampling.normal import SobolQMCNormalSampler

Multiple_tensor=torch.ones(2).reshape(1, 2).double().to("cuda")

print("Multiple_tensor.size(): ", Multiple_tensor.size())
print("Multiple_tensor.device: ", Multiple_tensor.device)


if Multiple_tensor.ndim == 1:
    Multiple_tensor = Multiple_tensor.unsqueeze(-1)

z_tensor =torch.zeros(256).reshape(1,  256).double().to("cuda")
print("Multiple_tensor.shape:", Multiple_tensor.shape)  # e.g., [160, M]
print("z_tensor.shape:", z_tensor.shape)  # [160, 64]
print("z_tensor.device:", z_tensor.device)

# ---- GPモデリング ----
M = Multiple_tensor.shape[1]
print("M: ", M)
gp_models = []
for i in range(M):
    y_i = standardize(Multiple_tensor[:, i]).unsqueeze(-1)  # [N, 1]
    gp_models.append(SingleTaskGP(z_tensor, y_i))

gp = ModelListGP(*gp_models).cuda()
mll = SumMarginalLogLikelihood(gp.likelihood, gp)

try:
    fit_gpytorch_mll(mll)
except Exception as e:
    print("error on fit_gpytorch_mll(mll):", e)

# ---- 取得関数定義 ----
ref_point = Multiple_tensor.min(dim=0)[0] - 0.1  # 各目的ごとにref_point設定
print("ref_point.device: ", ref_point.device)
partitioning = NondominatedPartitioning(Y=Multiple_tensor, ref_point=ref_point)

bounds = torch.stack([-10*torch.ones(1).reshape(1, 1), 10*torch.ones(1).reshape(1, 1)]).double().to("cuda")
print("bounds.device: ", bounds.device)

# acq_func = qLogExpectedHypervolumeImprovement(
acq_func = qLogNoisyExpectedHypervolumeImprovement(
    model=gp,
    ref_point=ref_point.tolist(),
    X_baseline = z_tensor.double(),
    sampler = SobolQMCNormalSampler(sample_shape=torch.Size([128])),
    prune_baseline=True,  # prune baseline points that have estimated zero probability of being Pareto optimal,
    constraints=[distance_constraint],

)
variable_q = 8

n_trial=1

list_smiles=[]
list_smiles_success=[]
# ---- 最適化ループ ----
for each_trial in range(n_trial):
    print(f"Loop {each_trial + 1} starts")
    try:
        candidate, acq_value = optimize_acqf(
            acq_function=acq_func,
            bounds=torch.stack([torch.zeros(z_tensor.shape[1]), torch.ones(z_tensor.shape[1])]).double().to("cuda"),
            q=variable_q ,
            num_restarts=5,
            raw_samples=10,
        )

        unnormalized_candidate = unnormalize(candidate, bounds)


        Multiple_val, each_smiles_list = obj_func(unnormalized_candidate, model, variable_q)


        if Multiple_val.ndim == 1:
            Multiple_val = Multiple_val.unsqueeze(-1)

        z_tensor = torch.cat([z_tensor, unnormalized_candidate.squeeze(0)]).to("cuda")
        Multiple_tensor = torch.cat([Multiple_tensor, Multiple_val.reshape(variable_q, 2)], dim=0).to("cuda")
        print("Multiple_tensor.shape: ", Multiple_tensor.shape)
        list_smiles.extend(each_smiles_list)

        each_smiles_list_filtered_success, each_smiles_list_filtered_fail, each_smiles_list_filtered_fail_idx = filter_valid_with_timeout(each_smiles_list)
        list_smiles_success.extend(each_smiles_list_filtered_success)

        # 無限値チェック
        if not torch.isfinite(Multiple_tensor).all():
            print("NaN or Inf detected, skipping this trial")
            continue


        # ---- モデル再学習 ----
        gp_models = []
        for i in range(M):
            y_i = standardize(Multiple_tensor[:, i]).unsqueeze(-1)
            gp_models.append(SingleTaskGP(z_tensor, y_i))

        gp = ModelListGP(*gp_models)
        mll = SumMarginalLogLikelihood(gp.likelihood, gp)

        try:
            fit_gpytorch_mll(mll)
        except Exception as e:
            print("error on fit_gpytorch_mll(mll):", e)

        # ---- 取得関数更新 ----
        bounds = torch.stack([z_tensor.min(dim=0)[0], z_tensor.max(dim=0)[0]])
        ref_point = Multiple_tensor.min(dim=0)[0] - 0.1
        partitioning = NondominatedPartitioning(Y=Multiple_tensor, ref_point=ref_point)
        acq_func = qLogNoisyExpectedHypervolumeImprovement(
            model=gp,
            ref_point=ref_point.tolist(),
            X_baseline = z_tensor,
            sampler = SobolQMCNormalSampler(sample_shape=torch.Size([128])),
            prune_baseline=True,  # prune baseline points that have estimated zero probability of being Pareto optimal
            # partitioning=partitioning,
        )
        del Multiple_val, unnormalized_candidate, candidate, ref_point

        torch.cuda.memory_cached()
    except Exception as e:
        print("NG")
        print(e)
        continue
    print(f"End of trial {each_trial + 1}")

Multiple_tensor.size():  torch.Size([1, 2])
Multiple_tensor.device:  cuda:0
Multiple_tensor.shape: torch.Size([1, 2])
z_tensor.shape: torch.Size([1, 256])
z_tensor.device: cuda:0
M:  2
ref_point.device:  cuda:0
bounds.device:  cuda:0
Loop 1 starts
dec_smiles:  ['CN1C(=O)c2cc3c(c(-c4c5c(c(-c6c7c(c(-c8c9c(c(-c%10c%11c(c(-c%12c%13c(c(-c%14c%15c(c(-c%16c%17c(c(-c%18c%19c(c(-c%20c%21c(c(-c%22c%23c(c(-c%24c%25c(c(-c%26c%27c(c(-c%28c%29c(c(-c%30c%31c(c(-c%32c%33c(c(-c%34c%35c(c(-c%36c%37c(c(-c%38c%39c(c(-c%40c%41c(c(-c%42c%43c(c(-c%44c%45c(c(-c%46c%47c(c(-c%48c%49c(c(-c%50c%51c(c(-c%52c%53c(c(-c%54c%55c(c(-c%56c%57c(c(-c%58c%59c(c(-c%60c%61c(cc%62c%60C(=O)C(C)(C)C%62O)C(=O)N(C)C%61=O)c%60c%58C(=O)C(C)(C)C%60O)C(=O)N(C)C%59=O)c%58c%56C(=O)C(C)(C)C%58O)C(=O)N(C)C%57=O)c%56c%54C(=O)C(C)(C)C%56O)C(=O)N(C)C%55=O)c%54c%52C(=O)C(C)(C)C%54O)C(=O)N(C)C%53=O)c%52c%50C(=O)C(C)(C)C%52O)C(=O)N(C)C%51=O)c%50c%48C(=O)C(C)(C)C%50O)C(=O)N(C)C%49=O)c%48c%46C(=O)C(C)(C)C%48O)C(=O)N(C)C%47=O)c%46c%44C(=O)C(C)(C)

Failed to patch pandas - unable to change molecule rendering


Found 30 ring systems
Motif: c1cc2cc3sccc3cc2s1  (count=30, atoms=[0, 1, 2, 3, 352, 353, 354, 355, 356, 357, 358, 359])
smiles_fragments:  c1cc2cc3sccc3cc2s1
success_list:  ['c1c2c(cc3c1CNC3)CCC2', 'C', 'C', 'c1cc2cc3sccc3cc2s1', 'C', 'c1cc2cc3sccc3cc2s1', 'C1=CSCC1', 'c1cc2cc3sccc3cc2s1']
fail_smiles_list_Nalpha []
Volume_list_test [157.44800000000004]
false_list_test []


Failed to patch pandas - unable to change molecule rendering
Failed to patch pandas - unable to change molecule rendering


0    2.258842
dtype: float64
0    142.712044
dtype: float64
Volume_list_test [28.784000000000006]
false_list_test []
0    0.25118
dtype: float64
0    17.511638
dtype: float64
Volume_list_test [28.736000000000008]
false_list_test []


...

GPU計算時間: 18.68秒
converged SCF energy = -547.921864543301


geometric-optimize called with the following command line:
/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py -f /root/.local/share/jupyter/runtime/kernel-010f6a21-b892-41de-b6ba-3ab3f0afa510.json

                                        ())))))))))))))))/                     
                                    ())))))))))))))))))))))))),                
                                *)))))))))))))))))))))))))))))))))             
                        #,    ()))))))))/                .)))))))))),          
                      #%%%%,  ())))))                        .))))))))*        
                      *%%%%%%,  ))              ..              ,))))))).      
                        *%%%%%%,         ***************/.        .)))))))     
                #%%/      (%%%%%%,    /*********************.       )))))))    
              .%%%%%%#      *%%%%%%,  *******/,     **********,      .))))))   
                .%%%%%%/      *%%%%%%,  **              ********    

SCF calculation completed successfully.
Number of occupied orbitals: 23
GAP:  [6.53465384]
GAP type:  <class 'numpy.ndarray'>
smiles_chosen:  c1cc2cc3sccc3cc2s1
使用するXYZデータ:
 C      3.405427    0.495863    0.761687
C      2.817129   -0.388367   -0.112160
C      1.386366   -0.278809   -0.136694
C      0.423207   -0.983730   -0.876993
C     -0.940116   -0.707450   -0.740616
S     -2.251656   -1.464687   -1.560322
C     -3.405427   -0.495863   -0.761687
C     -2.817129    0.388368    0.112160
C     -1.386366    0.278809    0.136694
C     -0.423207    0.983731    0.876993
C      0.940115    0.707449    0.740617
S      2.251655    1.464683    1.560325
H      4.459379    0.619399    0.969588
H      3.383538   -1.088997   -0.713371
H      0.746951   -1.758559   -1.568599
H     -4.459379   -0.619397   -0.969591
H     -3.383538    1.088995    0.713374
H     -0.746950    1.758563    1.568595

WARN: Mole.unit (angstrom) is changed to Bohr



Step    0 : Gradient = 3.107e-02/5.959e-02 (rms/max) Energy = -1167.5609060004
Hessian Eigenvalues: 2.30000e-02 2.30000e-02 2.30000e-02 ... 4.60031e-01 4.89360e-01 4.89634e-01
Step    1 : Displace = 9.331e-02/1.607e-01 (rms/max) Trust = 1.000e-01 (=) Grad = 2.069e-02/3.963e-02 (rms/max) E (change) = -1167.5637972072 (-2.891e-03) Quality = 0.114
Hessian Eigenvalues: 2.30000e-02 2.30000e-02 2.30000e-02 ... 4.86518e-01 4.89383e-01 6.83127e-01
Step    2 : Displace = 4.924e-02/8.577e-02 (rms/max) Trust = 4.666e-02 (-) Grad = 8.633e-03/1.641e-02 (rms/max) E (change) = -1167.5761584412 (-1.236e-02) Quality = 0.893
Hessian Eigenvalues: 2.30000e-02 2.30000e-02 2.30000e-02 ... 4.87467e-01 4.89091e-01 7.58804e-01
Step    3 : Displace = 2.734e-02/5.225e-02 (rms/max) Trust = 6.598e-02 (+) Grad = 2.969e-03/5.568e-03 (rms/max) E (change) = -1167.5782046229 (-2.046e-03) Quality = 0.791
Hessian Eigenvalues: 2.30000e-02 2.30000e-02 2.30000e-02 ... 4.88760e-01 4.92742e-01 6.96537e-01
Step    4 : Displace

GPU計算時間: 24.87秒
converged SCF energy = -1167.55696867553
SCF calculation completed successfully.
Number of occupied orbitals: 49
GAP:  [4.66325129]
GAP type:  <class 'numpy.ndarray'>
list_GAP:  [array([6.19228468]), array([22.32075793]), array([22.32075793]), array([4.66325129]), array([22.32075793]), array([4.66325129]), array([6.53465384]), array([4.66325129])]
_GAP_tensor:  tensor([[ 6.1923],
        [22.3208],
        [22.3208],
        [ 4.6633],
        [22.3208],
        [ 4.6633],
        [ 6.5347],
        [ 4.6633]], dtype=torch.float64)
Multiple_tensor.shape:  torch.Size([9, 2])
before_filter:  [['CN1C(=O)c2cc3c(c(-c4c5c(c(-c6c7c(c(-c8c9c(c(-c%10c%11c(c(-c%12c%13c(c(-c%14c%15c(c(-c%16c%17c(c(-c%18c%19c(c(-c%20c%21c(c(-c%22c%23c(c(-c%24c%25c(c(-c%26c%27c(c(-c%28c%29c(c(-c%30c%31c(c(-c%32c%33c(c(-c%34c%35c(c(-c%36c%37c(c(-c%38c%39c(c(-c%40c%41c(c(-c%42c%43c(c(-c%44c%45c(c(-c%46c%47c(c(-c%48c%49c(c(-c%50c%51c(c(-c%52c%53c(c(-c%54c%55c(c(-c%56c%57c(c(-c%58c%59c(c(-c%60c%61c(cc%6

In [ ]:
# aa

In [ ]:
type(np.array(uni_fragments)[3])

numpy.str_

In [ ]:
import os
os.getcwd()

'/content/drive/try_it_to_valify/try_it_to_valify_logs'

In [ ]:

target_tensor = Multiple_tensor.clone()
target_array = target_tensor.to("cpu").numpy()

import pandas as pd
result_collecter = pd.DataFrame([])
result_collecter["nD"] = pd.DataFrame(target_array[1:, 0])
result_collecter["GAP"] = pd.DataFrame(target_array[1:, 1])
result_collecter["smiles"] = pd.DataFrame(list_smiles)
result_collecter["smiles_success"] = pd.DataFrame(list_smiles_success)
result_collecter

,nD,GAP,smiles,smiles_success
0,1.648424,6.192285,CN1C(=O)c2cc3c(c(-c4c5c(c(-c6c7c(c(-c8c9c(c(-c...,c1c2c(cc3c1CNC3)CCC2
1,1.573836,22.320758,[CH2][CH][CH][CH][CH][CH][CH][CH][CH][CH][CH][...,C
2,1.575779,22.320758,CC1(C)COc2cn(-n3cc4c(c3)OCC(C)(C)CO4)cc2OC1,C
3,1.822868,4.663251,c1cc2cc3sc(-c4cc5cc6sccc6c(-c6cc7cc8sccc8c(-c8...,c1cc2cc3sccc3cc2s1
4,1.575454,22.320758,Cn1c2c(c3c1c1c(n3C)=NC=[SH]1)[SH]=CN=2,C
5,1.813151,4.663251,Fc1c2cc(-c3cc4c(F)c5scc(-c6cc7c(F)c8scc(-c9cc%...,c1cc2cc3sccc3cc2s1
6,1.686064,6.534654,C1=CSC(C2=CSC(C3=CSC(C4=CSC(C5=CSC(C6=CSC(C7=C...,C1=CSCC1
7,1.822208,4.663251,c1cc2c(-c3cc4c(-c5cc6c(-c7cc8c(-c9cc%10c(-c%11...,c1cc2cc3sccc3cc2s1


In [ ]:
import datetime
result_collecter.to_csv(f"result_collecter_{datetime.date.today()}_{experiment_number}_FRATTVAE.csv")

In [ ]:
from google.colab import runtime
runtime.unassign()